<a href="https://colab.research.google.com/github/Cath-Strategic-Tech/adpdx_etl/blob/main/ADPDX_ClergyDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Introduction

The following notebook orchestrates the migration of ADPDX Accounts into Salesforce.


# Order of Loading

1. Vicariates
1. Organizations [MANUAL]
1. Religious Parents
1. Religious Communities
1. Religious Superiors
1. Contacts
1. Contact > Register Entries
1. Contact > Education Affiliations [MANUAL]
1. Contact > Ecclesial Affiliations [MANUAL]
1. Affiliations [MANUAL]


# Order of Operations

- Setup Enviro

  - [DONE] UDFs
  - [DONE] Load SF xref data

- ACCOUNTS

  - Extract Source Data
    - [DONE] Load 6 tables into separate dataframes
    - [DONE] Merge into single accounts table
    - [DONE]: Fix the ExternalID so that it references the original table, not the AccRecordType
  - Transform
    - Strip phone numbers
    - Validate email addresses
    - TODO: handle churches that aren't parishes (missions, non-diocesan parishes, etc.)
  - Load
    - [DONE]Vicariates
    - [DONE] Organizations (Parishes, Schools, Newman Centres, Offices)
    - Religious
      - [DONE] Religious Parent accounts
      - [DONE] Religious Communities
      - [DONE] Religious Superiors (Contacts, set AccountID to Rel. Parent)
        - [DONE]: Handle invalid email addresses
        - TODO: Handle duplicate entries
      - TODO: Update Religious Communities with lookup to Rel. Superior
  - TODO: Unit Tests
    - Num of Accounts, by type
    - Spot checking 3-5 account records & field values

- CONTACTS

  - Extract

    - [DONE] Import Contact records
    - TODO: Get Photo directory @soames

  - Analysis

    - [DONE] Check columns & row count (3016)
    - [DONE] Identify unique languages

  - Transform

    - Complete ETL of fields that are more complex (search for TODO)
    - [DONE] Create new df_contact_staging, renaming columns to SF APIs
    - [DONE] Drop columns that don't map to Contact
    - Migrate Languages field (waiting on next package version) @soames
      - TODO: transform `,` to `;` so imports to multi-select list correctly
    - TODO: Concat Mailing Street Address lines into one
    - TODO: Handle Private Addresses: decide if will code changes or NOT use a custom Private Address field.
    - [DONE] Update boolean fields to True/False
    - [DONE] Set Contact Record Type (UDF)
    - [DONE] Validate, drop invalid emails
    - [DONE] Generate ExternalID > 'Archdpdx_External_Id\_\_c'
    - TODO: Preferred Email/Phone > where blank, set a default. Currently, all are getting set to 'Personal' and 'Mobile.'
    - TODO: Ecclesial Status (not mapping correctly)
    - [DONE] DROP columns that haven't been mapped yet

  - Load
    - [DONE] Set JobID to curr_job_id
    - [DONE] Handle character encoding that is geting messed up

- CONTACTS > SPOUSES

- CONTACTS > PHOTOS

- CONTACTS > REGISTER ENTRIES

  - Parse columns into types of Sacraments or Notations
  - For lookups to Celebrants, query SF for contacts, create missing records
  - Generate External ID, apply to df
  - Clean up (remove extra columns, NaNs)
  - Upsert records

- CONTACTS > AFFILIATIONS

  - Map the various Contact fields that are actually Affiliations (start with manual migration)
    - Education/Degrees
    - Minor Orders
    - Religious Vows
    - Candidacy records (should this be another object?)
    - In/Excardination
    - Faculties

- AFFILIATIONS TABLE

  - Extract

    - [DONE] Turn the 'Org Table Name' & 'org Table Link' columns into External ID
    - Map in the Account IDs from SF

  - Transform

    - Parse RecordTypeId
    - Parse Category
    - Map columns to SF field APIs

  - Load


# Setup Enviro


In [128]:
# !conda install -y simple-salesforce
# !conda install -y email_validator
# !conda install -y python-dotenv
# !conda install import-ipynb


In [129]:
# enviro setup

import pandas as pd
import numpy as np

from datetime import datetime
now = datetime.now()

from simple_salesforce import Salesforce

In [130]:
# import environment variables (SF login credentials)
from dotenv import load_dotenv
import os

load_dotenv()

True

In [131]:
# @title Global Variables { run: "auto", vertical-output: true, display-mode: "both" }

target_enviro = "adpdx_devpro" # @param {type:"string"}

# @markdown The `run_upserts` variable controls whether or not upserts to Salesforce are executed when the notebook is run.
run_upserts = "True" # @param ["True", "False"]

In [132]:
# ADPDX dev_pro credentials
adpdx_user = os.getenv('ADPDX_UAT_USER')
print(adpdx_user)
adpdx_pass = os.getenv('ADPDX_UAT_PASS')
print( adpdx_pass)
adpdx_token = os.getenv('ADPDX_UAT_TOKEN')
print(adpdx_token)

# instantiate a SF session object
sf = Salesforce(domain='test', username=adpdx_user, password=adpdx_pass, security_token=adpdx_token)

matt+adpdx@meribahflow.com.uat
8n&ycaQJ
aKRgyLyAX5V0YPeJJRX5bDdi


## UDFs


In [133]:
from simple_salesforce import Salesforce

# Job ID Incrementer
def update_job_id(file_name):
    # Open the file in read mode and get the current job ID
    with open(file_name, 'r') as file:
        current_job_id = int(file.readline())

    # Increment the job ID
    new_job_id = current_job_id + 1

    # Open the file in write mode and update the job ID
    with open(file_name, 'w') as file:
        file.write(str(new_job_id))

    # Return the new job ID
    return new_job_id


# Concates two DF columns for an External ID
def concat_columns(df, columns, new_column, separator='_'):
    """
    Concatenates the values from specified columns into a single string
    with the specified separator and populates a new column in the DataFrame.

    Args:
    - df: pandas DataFrame
    - columns: list of column names to concatenate
    - new_column: name of the new column to be created
    - separator: separator to use between concatenated values (default is '_')

    Returns:
    - Updated pandas DataFrame with the new column
    """
    df[new_column] = df[columns].astype(str).apply(lambda x: separator.join(x), axis=1)
    return df


# Gets or creates a Diocesan account based on the Account Name
def get_or_create_diocesan_account(sf, account_name):
    """
    Searches for an account by name, returns the ID if found,
    otherwise creates the account with RecordType 'Church' and 'mbfc__Church_Type__c' set to 'Diocese',
    and then returns the new ID.

    Parameters:
    sf (Salesforce): Salesforce connection object
    account_name (str): The name of the account to search for or create

    Returns:
    str: The ID of the found or created account
    """

    # Query for the Record Type ID using the Developer Name 'Church'
    record_type_query = "SELECT Id FROM RecordType WHERE SobjectType = 'Account' AND DeveloperName = 'Church' LIMIT 1"
    record_type_result = sf.query(record_type_query)
    if record_type_result['records']:
        record_type_id = record_type_result['records'][0]['Id']
    else:
        raise ValueError("No RecordType found with DeveloperName 'Church'")

    # Search for the Account by name
    account_query = f"SELECT Id FROM Account WHERE Name = '{account_name}' LIMIT 1"
    account_result = sf.query(account_query)
    
    if account_result['records']:
        # Account found, return the ID
        return account_result['records'][0]['Id']
    else:
        # Account not found, create a new Account
        account_data = {
            'Name': account_name,
            'RecordTypeId': record_type_id,
            'mbfc__Church_Type__c': 'Diocese'
        }
        new_account = sf.Account.create(account_data)
        return new_account['id']
    
    from simple_salesforce import Salesforce

# improved version of the get_or_create_diocesan_account function
def get_or_create_account(sf, account_name, record_type_dev_name, church_type):
    """
    Searches for an account by name, returns the ID if found,
    otherwise creates the account with the specified Record Type and Church Type,
    and then returns the new ID.

    Parameters:
    sf (Salesforce): Salesforce connection object
    account_name (str): The name of the account to search for or create
    record_type_dev_name (str): The developer name of the Record Type to use for creating the account
    church_type (str): The Church Type to set for the new account

    Returns:
    str: The ID of the found or created account
    """

    # Query for the Record Type ID using the provided developer name
    record_type_query = f"SELECT Id FROM RecordType WHERE SobjectType = 'Account' AND DeveloperName = '{record_type_dev_name}' LIMIT 1"
    record_type_result = sf.query(record_type_query)
    if record_type_result['records']:
        record_type_id = record_type_result['records'][0]['Id']
    else:
        raise ValueError(f"No RecordType found with DeveloperName '{record_type_dev_name}'")

    # Search for the Account by name
    account_query = f"SELECT Id FROM Account WHERE Name = '{account_name}' LIMIT 1"
    account_result = sf.query(account_query)
    
    if account_result['records']:
        # Account found, return the ID
        return account_result['records'][0]['Id']
    else:
        # Account not found, create a new Account
        account_data = {
            'Name': account_name,
            'RecordTypeId': record_type_id,
            'mbfc__Church_Type__c': church_type
        }
        new_account = sf.Account.create(account_data)
        return new_account['id']

# Example usage
# sf = Salesforce(username='your_username', password='your_password', security_token='your_security_token')
# account_id = get_or_create_account(sf, 'Diocese of Calgary', 'Church', 'Diocese')
# print(f"Account ID: {account_id}")

In [134]:
# Add a Salesforce record ID column to a DataFrame based on matching external ID field values

import pandas as pd
from simple_salesforce import Salesforce
from simple_salesforce.exceptions import SalesforceMalformedRequest, SalesforceError

def add_salesforce_record_ids(sf, dataframe, df_column_name, sf_object_name, sf_external_id_field, new_column_name, chunk_size=1000):
    """
    Add a Salesforce record ID column to a DataFrame based on matching external ID field values.

    Parameters:
    sf (Salesforce): The Salesforce connection instance.
    dataframe (pd.DataFrame): The pandas DataFrame containing data to match.
    df_column_name (str): The column name in the DataFrame to match with Salesforce.
    sf_object_name (str): The Salesforce object name (e.g., 'Contact').
    sf_external_id_field (str): The external ID field in Salesforce to match.
    new_column_name (str): The name for the new DataFrame column to hold Salesforce record IDs.
    chunk_size (int): The number of records to include in each chunk for querying Salesforce.

    Returns:
    pd.DataFrame: The original DataFrame with the new column containing Salesforce record IDs.
    """
    # Ensure the dataframe column name exists in the dataframe
    if df_column_name not in dataframe.columns:
        raise ValueError(f"Column '{df_column_name}' not found in DataFrame.")
    
    # Create a set of unique values from the specified DataFrame column
    unique_values = dataframe[df_column_name].dropna().unique()
    
    id_mapping = {}
    
    # Process the unique values in chunks
    for start in range(0, len(unique_values), chunk_size):
        chunk_values = unique_values[start:start + chunk_size]
        chunk_values_str = ", ".join([f"'{val}'" for val in chunk_values])
        
        soql_query = f"SELECT Id, {sf_external_id_field} FROM {sf_object_name} WHERE {sf_external_id_field} IN ({chunk_values_str})"
        
        try:
            query_result = sf.query_all(soql_query)
        except SalesforceMalformedRequest as e:
            raise ValueError(f"Malformed request error: {e.content}")
        except SalesforceError as e:
            raise ValueError(f"Salesforce error: {e.content}")
        
        # Update the id_mapping with results from the current chunk
        id_mapping.update({record[sf_external_id_field]: record['Id'] for record in query_result['records']})
    
    # Map the Salesforce record IDs to the DataFrame
    dataframe[new_column_name] = dataframe[df_column_name].map(id_mapping)
    
    return dataframe

In [135]:
import pandas as pd
from simple_salesforce import Salesforce
from simple_salesforce.exceptions import SalesforceMalformedRequest, SalesforceError

def find_salesforce_record_id(sf, df, column_to_search, sf_object_name, sf_field_name, new_column_name, match_behavior='first'):
    """
    Find Salesforce record IDs for a DataFrame column and add a new column with the Salesforce record IDs.

    Parameters:
    sf (Salesforce): The Salesforce connection instance.
    df (pd.DataFrame): The pandas DataFrame containing data.
    column_to_search (str): The column name in the DataFrame to search against Salesforce.
    sf_object_name (str): The Salesforce object name (e.g., 'Contact').
    sf_field_name (str): The field name in Salesforce to match.
    new_column_name (str): The name for the new DataFrame column to hold Salesforce record IDs.
    match_behavior (str): Behavior when multiple matches found ('first' or 'alert').

    Returns:
    pd.DataFrame: The original DataFrame with the new column containing Salesforce record IDs.
    """
    if column_to_search not in df.columns:
        raise ValueError(f"Column '{column_to_search}' not found in DataFrame.")

    df[new_column_name] = None
    multiple_matches_found = False

    unique_values = df[column_to_search].dropna().unique()
    chunk_size = 1000  # Adjust chunk size as needed

    for start in range(0, len(unique_values), chunk_size):
        chunk_values = unique_values[start:start + chunk_size]
        chunk_values_str = ", ".join([f"'{val}'" for val in chunk_values])

        soql_query = f"SELECT Id, {sf_field_name} FROM {sf_object_name} WHERE {sf_field_name} IN ({chunk_values_str})"
        
        try:
            query_result = sf.query_all(soql_query)
        except SalesforceMalformedRequest as e:
            raise ValueError(f"Malformed request error: {e.content}")
        except SalesforceError as e:
            raise ValueError(f"Salesforce error: {e.content}")

        id_mapping = {}
        for record in query_result['records']:
            key = record[sf_field_name]
            if key in id_mapping:
                multiple_matches_found = True
                if match_behavior == 'first':
                    continue  # Skip subsequent matches if 'first' behavior is selected
            id_mapping[key] = record['Id']

        df[new_column_name] = df[column_to_search].map(id_mapping)

    if multiple_matches_found and match_behavior == 'alert':
        print("Alert: Multiple matches found for some records.")

    return df

# Example usage
# df_contact_staging = find_salesforce_record_id(sf, df_contact_staging, 'Link_to_Religious_Community', 'Contact', 'Archdpdx_Migration_Id__c', 'New_Column_Name', match_behavior='alert')


In [136]:
# Upsert functions

import pandas as pd
import numpy as np
from simple_salesforce import Salesforce, SalesforceMalformedRequest, SalesforceError
from datetime import datetime, date

def convert_non_serializables(data):
    """Convert non-serializable objects to serializable formats."""
    for key, value in data.items():
        try:
            if isinstance(value, (datetime, date)):
                data[key] = value.isoformat()
            elif isinstance(value, float) and np.isnan(value):
                data[key] = None
            elif pd.isna(value):
                data[key] = None
            elif isinstance(value, (int, bool, str)):
                data[key] = value
            else:
                data[key] = str(value)  # Convert other types to string
        except Exception as e:
            print(f"Error processing key: {key}, value: {value}, error: {e}")
    return data


def upsert_to_salesforce(sf, dataframe, object_name, external_id_field):
    """
    Upsert records to Salesforce from a pandas DataFrame.

    Parameters:
    sf (Salesforce): The Salesforce connection instance.
    dataframe (pd.DataFrame): The pandas DataFrame containing data to upsert.
    object_name (str): The Salesforce object name (e.g., 'Contact').
    external_id_field (str): The external ID field used for upserts.
    """
    successful_upserts = 0
    failed_upserts = 0

    # Replace placeholder values with None in the DataFrame
    dataframe.replace({None: pd.NA, ' ': None, '': None}, inplace=True)

    # Convert DataFrame to a list of dictionaries
    data_to_upsert = dataframe.to_dict(orient='records')

    for data in data_to_upsert:
        try:
            data = convert_non_serializables(data)
            external_id = data.pop(external_id_field)

            # Perform upsert using only the External ID
            response = getattr(sf, object_name).upsert(f'{external_id_field}/{external_id}', data)
            successful_upserts += 1
            print(f"Successfully upserted {object_name} with External ID: {external_id}")
        except SalesforceMalformedRequest as e:
            failed_upserts += 1
            print(f"Malformed request error when upserting {object_name} with External ID: {external_id}. Error: {e.content}")
        except SalesforceError as e:
            failed_upserts += 1
            print(f"Salesforce error when upserting {object_name} with External ID: {external_id}. Error: {e.content}")
        except Exception as e:
            failed_upserts += 1
            print(f"Failed to upsert {object_name} with External ID: {external_id}. Error: {e}")

    print(f"Upsert completed. Successful upserts: {successful_upserts}, Failed upserts: {failed_upserts}")




def convert_non_serializables(record):
    """Convert non-serializable values to strings or handle them appropriately."""
    for key, value in record.items():
        if pd.isna(value):
            record[key] = None
        elif isinstance(value, pd.Timestamp):
            record[key] = value.isoformat()
        elif isinstance(value, (pd.Timedelta, pd.Period)):
            record[key] = str(value)
    return record

def upsert_to_salesforce_bulk(sf, dataframe, object_name, external_id_field, failed_log_file, batch_size=10000):
    """
    Upsert records to Salesforce from a pandas DataFrame using the Bulk API.

    Parameters:
    sf (Salesforce): The Salesforce connection instance.
    dataframe (pd.DataFrame): The pandas DataFrame containing data to upsert.
    object_name (str): The Salesforce object name (e.g., 'Contact').
    external_id_field (str): The external ID field used for upserts.
    failed_log_file (str): The file name where failed upsert records will be logged.
    batch_size (int): The number of records to include in each batch.
    """
    successful_upserts = 0
    failed_upserts = 0

    # Replace placeholder values with None in the DataFrame
    dataframe.replace({None: pd.NA, ' ': None, '': None}, inplace=True)

    # Convert DataFrame to a list of dictionaries
    data_to_upsert = dataframe.to_dict(orient='records')

    with open(failed_log_file, 'a') as log_file:
        # Process data in batches
        for i in range(0, len(data_to_upsert), batch_size):
            batch_data = data_to_upsert[i:i + batch_size]
            batch_data = [convert_non_serializables(record) for record in batch_data]

            try:
                # Perform bulk upsert
                response = sf.bulk.__getattr__(object_name).upsert(batch_data, external_id_field=external_id_field)

                for res in response:
                    if res['success']:
                        successful_upserts += 1
                    else:
                        failed_upserts += 1
                        log_file.write(f"Failed to upsert record with External ID: {res['external_id']}. Errors: {res['errors']}\n")

            except SalesforceMalformedRequest as e:
                failed_upserts += len(batch_data)
                log_file.write(f"Malformed request error: {e.content}\n")
            except SalesforceError as e:
                failed_upserts += len(batch_data)
                log_file.write(f"Salesforce error: {e.content}\n")
            except Exception as e:
                failed_upserts += len(batch_data)
                log_file.write(f"Unexpected error: {e}\n")

    print(f"Bulk upsert completed. Successful upserts: {successful_upserts}, Failed upserts: {failed_upserts}")

## Extract Salesforce xref data

The following cells downloads all records from the target Salesforce enviro for the following objects:

- RecordTypes
- Users
- Accounts
- Contacts


In [137]:
# Get or create the Diocesan Account and get its ID

# calls old function
# diocesan_account_id = get_or_create_diocesan_account(sf, 'Archdiocese of Portland in Oregon')

# calls new function
diocesan_account_id = get_or_create_account(sf, 'Archdiocese of Portland in Oregon', 'Church', 'Diocese')

print(f"Account ID: {diocesan_account_id}")

Account ID: 001Dx00001HwDsgIAF


In [138]:
# get all ACTIVE SF users

sf_users = sf.query('Select Alias, FirstName, LastName, Username, id from User WHERE IsActive = True')
df_sf_users = pd.DataFrame(sf_users['records'])
df_sf_users = df_sf_users.drop(columns = 'attributes')
df_sf_users.shape

(16, 5)

In [139]:
# get all SF Record Types
get_all_recordTypes = 'Select Id, Name, DeveloperName, sObjecttype, namespaceprefix from RecordType'

# get list of records, add to dataframe
sf_recordTypes = sf.query(get_all_recordTypes)
df_sf_recordTypes = pd.DataFrame(sf_recordTypes['records'])
df_sf_recordTypes = df_sf_recordTypes.drop(columns = 'attributes')

# Create a dictionary mapping 'DeveloperName' to 'Id' for faster lookup
record_types_mapping = df_sf_recordTypes.set_index('DeveloperName')['Id'].to_dict()

df_sf_recordTypes

,Id,Name,DeveloperName,SobjectType,NamespacePrefix
0,012Dx0000003p4xIAA,Church,Church,Account,mbfc
1,012Dx0000003p4yIAA,Deanery,Deanery,Account,mbfc
2,012Dx0000003p4zIAA,Group,Group,Account,mbfc
3,012Dx0000003p50IAA,Organization,Organization,Account,mbfc
4,012Dx0000003p51IAA,Property,Property,Account,mbfc
5,012Dx0000003p52IAA,Religious,Religious,Account,mbfc
6,012Dx0000003p53IAA,z) All Types,All_Types,mbfc__Affiliation__c,mbfc
7,012Dx0000003p54IAA,Any,Any,mbfc__Affiliation__c,mbfc
8,012Dx0000003p55IAA,Pastoral Assignments,Assignments_Clergy,mbfc__Affiliation__c,mbfc
9,012Dx0000003p56IAA,Chancery Users,Chancery_Users,mbfc__Affiliation__c,mbfc


In [140]:
# get SF Account
get_all_accounts = 'Select id, Name, RecordTypeId, Type, mbfc__Parish_Code__c, Job_Id__c, Archdpdx_Migration_Id__c from Account'

# get list of records, add to dataframe
sf_accounts = sf.query(get_all_accounts)
df_sf_accounts = pd.DataFrame(sf_accounts['records'])
df_sf_accounts = df_sf_accounts.drop(columns = 'attributes')
df_sf_accounts.shape

(2000, 7)

In [141]:
# get SF Contacts
get_all_contacts = 'Select id, Name, npe01__Type_of_Account__c, RecordTypeId, Archdpdx_Migration_Id__c, CreatedById from Contact'

# get list of records, add to dataframe
sf_contacts = sf.query(get_all_contacts)
df_sf_contacts = pd.DataFrame(sf_contacts['records'])
df_sf_contacts = df_sf_contacts.drop(columns = 'attributes')
df_sf_contacts.shape

(2000, 6)

# ACCOUNTS


## Extract


### Load ArchdPDX csvs as DataFrames

ADPDX data for organizations is held in 6 tables, all of which will be migrated into Salesforce's Accounts object.


In [142]:
df_offices = pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Offices.csv', skiprows= lambda x: x in [1])
df_offices["src_table"] = 'Offices'
df_offices["AccountRecordType"] = 'Organization'
df_offices.rename({
    "Common Name": "Name",
    "Name": "Formal_Name__c"
    }, axis="columns", inplace=True)


In [143]:
df_parishes = pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Parishes (3).csv', dtype={'Vicariate': 'object', 'Established': 'str', 'Mission Of': 'str'}, skiprows= lambda x: x in [1])
df_parishes["src_table"] = 'Parishes'
df_parishes["AccountRecordType"] = 'Church'
# df_parishes.rename({"Parish Formal Name": "Account Name"}, axis="columns", inplace=True)
df_parishes.rename({
                    "Parish Formal Name": "Formal_Name__c",
                    "Common Name": "Name"
                }, axis="columns", inplace=True)


In [144]:
df_religious = pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/RelCommunities.csv', skiprows= lambda x: x in [1])
df_religious["src_table"] = 'RelCommunities'
df_religious["AccountRecordType"] = 'Religious'
df_religious.rename({
                    "Community Name": "Formal_Name__c",
                    "Common Name": "Name"
                     }, axis="columns", inplace=True)


In [145]:
df_schools = pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Schools.csv', skiprows= lambda x: x in [1])
df_schools["src_table"] = 'Schools'
df_schools["AccountRecordType"] = 'Organization'
df_schools.rename({
                    "School Name": "Formal_Name__c",
                    "Common Name": "Name"
                    
                    }, axis="columns", inplace=True)

In [146]:
df_vicariates = pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Vicariates.csv', skiprows= lambda x: x in [1])
df_vicariates["src_table"] = 'Vicariates'
df_vicariates["AccountRecordType"] = 'Deanery'
# As we want to designate the Common Name as what will be the Account Name in Salesforce, we are renaming these columns in a different pattern than prior CSVs.
df_vicariates.rename({"Common Name": "Name"}, axis="columns", inplace=True)
df_vicariates

,Record Number,Name,Vicariate Name,Archdiocese Assigns Clergy,src_table,AccountRecordType
0,1,Albany-Corvallis Vicariate,Albany-Corvallis,Yes,Vicariates,Deanery
1,2,"Beaverton, Suburban Vicariate","Beaverton, Suburban",Yes,Vicariates,Deanery
2,3,Columbia County Vicariate,Columbia County,Yes,Vicariates,Deanery
3,4,Downtown Portland Vicariate,Downtown Portland,Yes,Vicariates,Deanery
4,5,"East Portland, Suburban Vicariate","East Portland, Suburban",Yes,Vicariates,Deanery
5,6,Marion County Vicariate,Marion County,Yes,Vicariates,Deanery
6,7,Metropolitan Eugene Vicariate,Metropolitan Eugene,Yes,Vicariates,Deanery
7,8,Metropolitan Salem Vicariate,Metropolitan Salem,Yes,Vicariates,Deanery
8,9,North Coast Vicariate,North Coast,Yes,Vicariates,Deanery
9,10,Northeast Portland Vicariate,Northeast Portland,Yes,Vicariates,Deanery


In [147]:
df_newman = pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/NewmanCenters.csv', skiprows= lambda x: x in [1])
df_newman["src_table"] = 'NewmanCenters'
df_newman["AccountRecordType"] = 'Organization'
df_newman.rename({
                    "Newman Center Name": "Formal_Name__c",
                    "Common Name": "Name",
                    "Newman Center City": "Mailing Address City2"
                  }, axis="columns", inplace=True)


Each of the 6 tables has an overlapping but distinct set of columns, making it challenging to conform these tables into a single staging table.

In addition, columns that correspond to the same field in salesforce are named differently in each table (eg. 'Parish City' vs. 'Religious City' vs. 'Newman Center City')


In [148]:
print('TABLE: (ROWS, COLUMNS)\n')

print(f'Offices:    {df_offices.shape}')
print(f'Parishes:   {df_parishes.shape}')
print(f'Religious:  {df_religious.shape}')
print(f'Schools:    {df_schools.shape}')
print(f'Vicariates: {df_vicariates.shape}')
print(f'Newman Ctr: {df_newman.shape}')

TABLE: (ROWS, COLUMNS)

Offices:    (35, 18)
Parishes:   (151, 45)
Religious:  (70, 34)
Schools:    (56, 26)
Vicariates: (18, 6)
Newman Ctr: (4, 37)


### Merge DFs into a single Accounts DF

This step takes 6 different tables and combines them into a single Accounts table for cleaning and staging.


In [149]:
# init list of DataFrames
src_accounts = [df_offices, df_parishes, df_religious, df_schools, df_vicariates, df_newman]

# concats the various Account dataframes into one large table
accounts = pd.concat(src_accounts, ignore_index=True)

In [150]:
accounts.columns

Index(['Record Number', 'Name', 'Formal_Name__c', 'Archdiocese Assigns Clergy',
       'Locator Description', 'Mailing Address', 'Mailing Address 2',
       'Mailing Address City', 'Mailing Address State',
       'Mailing Address Province', 'Mailing Address Postal Code',
       'Mailing Address Country', 'Phone', 'Fax', 'Email', 'Web Site',
       'src_table', 'AccountRecordType', 'Sort Name', 'Parish Name',
       'Parish City', 'Mission Of', 'Established', 'Vicariate', 'Non-Latin',
       'County', 'Disabled Access', 'Sanctuary Capacity',
       'Lat/Long Coordinates Decimal', 'Google Small Embed URL',
       'Miles to Pastoral Center', 'Schedule 1 Head', 'Schedule 1 Text',
       'Schedule 2 Head', 'Schedule 2 Text', 'Schedule 3 Head',
       'Schedule 3 Text', 'Schedule 4 Head', 'Schedule 4 Text',
       'Schedule 5 Head', 'Schedule 5 Text', 'Schedule 6 Head',
       'Schedule 6 Text', 'Schedule 7 Head', 'Schedule 7 Text',
       'Community City', 'Order Full Name', 'Order Common N

## Transform


Time to do some table column renaming and re-organizing!


In [151]:
# renames columns headers to consolidate account names into SF-conformed data model
accounts.rename({"Common Name": "Name, City"}, axis="columns", inplace=True)

accounts.rename(
    columns={
        # 'Account Name': 'Name',
        'Mailing Address': 'BillingStreet1',
        'Mailing Address 2': 'BillingStreet2',
        'Mailing Address City': 'BillingCity',
        'Mailing Address State': 'BillingState',
        'Mailing Address Postal Code': 'BillingPostalCode',
        'Mailing Address Country': 'BillingCountry',
        'Email': 'mbfc__Email__c',
        'Web Site': 'Website',
        'Order Common Name': 'mbfc__Abbreviation__c',
        'Order Letters': 'mbfc__Religious_Suffix__c',
        'Men or Women': 'mbfc__Type_Members__c',
        'Archdiocese Assigns Clergy': 'Archdiocese_Assigns_Clergy__c',
        'Locator Description': 'Locator_Description__c',
        'Mission Of': 'Parent_Parish__c',
        'Established': 'mbfc__Date_Established__c',
        'County': 'County__c',
        'Disabled Access': 'Disabled_Access__c',
        'Sanctuary Capacity': 'Sanctuary_Capacity__c',
        'Miles to Pastoral Centre': 'Miles_to_Pastoral_Centre__c',
        'Archdiocesan School Code': 'Archdiocesan_School_Code__c',
        'Grades Provided': 'Grades_Provided__c'

    },
    inplace=True
)


# reorder column order
col = accounts.pop('Name')
accounts.insert(2, col.name, col)

col = accounts.pop('Parish Name')
accounts.insert(3, col.name, col)

col = accounts.pop('AccountRecordType')
accounts.insert(1, col.name, col)



In [152]:
accounts[accounts.BillingStreet2.isna() == False]

,Record Number,AccountRecordType,Formal_Name__c,Name,Parish Name,Archdiocese_Assigns_Clergy__c,Locator_Description__c,BillingStreet1,BillingStreet2,BillingCity,...,Major Superior Email,School City,Parish Link,Vicariate Link,Archdiocesan_School_Code__c,Grades_Provided__c,Mailing Address 1,Mailing Address Zip,Vicariate Name,Mailing Address City2
14,32,Organization,Diaconate Office,Diaconate Office,NaN,Yes,NaN,Pastoral Center,2838 E Burnside St,Portland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,58,Organization,Office of Marketing and Communications,Office of Marketing and Communications,NaN,Yes,NaN,Pastoral Center,2838 E Burnside St,Portland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,1,Church,"Our Lady of Perpetual Help, St Mary’s","Our Lady of Perpetual Help, St Mary’s, Albany",NaN,Yes,SW Ellsworth St between 8th and 9th Streets,"Our Lady of Perpetual Help, St Mary’s Parish",815 Broadalbin St SW,Albany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,2,Church,St. Andrew Dũng-Lạc,"St. Andrew Dũng-Lạc Mission, Aloha",NaN,No,SW Grabhorn Rd/209th Ave and Farmington Rd,St. Andrew Dũng-Lạc Mission,7390 SW Grabhorn Rd,Aloha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,3,Church,St. Elizabeth Ann Seton,"St. Elizabeth Ann Seton, Aloha",NaN,Yes,NaN,St. Elizabeth Ann Seton Parish,3145 SW 192nd Ave,Aloha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,62,Religious,Work of Jesus the High Priest,"Work of Jesus the High Priest, Gresham (OJSS)",NaN,No,NaN,OJSS Community,451 NW 1st St,Gresham,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,64,Religious,Heralds of the Good News,"Heralds of the Good News, Portland (HGN)",NaN,No,NaN,c/o Chancellor,2838 E Burnside St,Portland,...,rkappumkal@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,65,Religious,Missionary Oblates of Mary Immaculate,"Missionary Oblates of Mary Immaculate, Rome, I...",NaN,No,NaN,Missionary Oblates of Mary Immaculate,Via Aurelia 290,Roma,...,gensec@omigen.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,73,Religious,Brothers of Saint John,"Brothers of Saint John, Laredo, TX (CSJ)",NaN,No,NaN,St. John Priory,505 Century Dr S,Laredo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
# merge two Non-Latin columns into one 
accounts['Non_Latin__c'] = accounts['Non-Latin'].combine_first(accounts['Non-Latin Rite']) 

In [154]:
# export merged tables DESCRIPTION to CSV for mapping
accounts.describe(include='all').transpose().to_csv(f'/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/working/accounts.csv')
accounts.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Record Number,334.0,NaN,NaN,NaN,54.5,41.389801,1.0,21.25,45.0,76.75,173.0
AccountRecordType,334,4,Church,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Formal_Name__c,316,273,St. Mary,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Name,334,334,Pastoral Center,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Parish Name,5,5,St. Anne,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
Mailing Address 1,56,55,4420 SW St Marys Dr,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mailing Address Zip,56.0,NaN,NaN,NaN,97222.446429,124.9586,97005.0,97134.75,97217.5,97301.0,97526.0
Vicariate Name,18,18,Albany-Corvallis,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mailing Address City2,4,4,Corvallis,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
# Create a single BillingAddress field

# Concatenate the two columns with CHAR(10) as separator
accounts['BillingStreet'] = accounts[['BillingStreet1', 'BillingStreet2']].apply(lambda x: '\n'.join(x.dropna()), axis=1)

# Drop the original columns
accounts.drop(columns=['BillingStreet1', 'BillingStreet2'], inplace=True)

In [156]:
# Handle boolean fields

boolean_columns_to_convert = [
    'Archdiocese_Assigns_Clergy__c', 
    'Non_Latin__c', 
    'Disabled_Access__c', 
    ]

# Convert 'Yes'/'No' to True/False
accounts[boolean_columns_to_convert] = accounts[boolean_columns_to_convert].replace({'Yes': True, 'No': False, None: False})



In [157]:
accounts[boolean_columns_to_convert].sample(10)

,Archdiocese_Assigns_Clergy__c,Non_Latin__c,Disabled_Access__c
74,True,False,True
58,True,False,True
241,False,False,False
151,True,False,True
304,True,False,False
158,True,False,True
242,False,False,False
171,True,False,True
172,True,False,False
101,True,False,True


In [158]:
# Religious Order fields > conform to new data model

# Apply logic to create new columns
accounts['Religious_Secular_Order__c'] = accounts.apply(
    lambda x: 'Religious Order' if x['Religious Order'] == 'Yes' else ('Secular Order' if x['Secular Order'] == 'Yes' else None), axis=1
)

accounts['Pontifical_or_Diocesan_Order__c'] = accounts.apply(
    lambda x: 'Diocesan Order' if x['Diocesan Order'] == 'Yes' else ('Pontifical Order' if x['Pontifical Order'] == 'Yes' else None), axis=1
)

accounts.drop(columns=['Religious Order', 'Secular Order', 'Diocesan Order', 'Pontifical Order'], inplace=True)

In [159]:
print(accounts['mbfc__Date_Established__c'].dtype)

object


In [160]:
# Handle Date fields that are only YYYY

# Ensure all values in 'mbfc__Date_Established__c' are strings
accounts['mbfc__Date_Established__c'] = accounts['mbfc__Date_Established__c'].astype(str)

# Define a function to transform valid year values
def transform_year(year):
    if pd.notna(year) and year.replace('.', '', 1).isdigit() and len(year.split('.')[0]) == 4:
        return pd.to_datetime(year.split('.')[0] + '-01-01')
    else:
        return pd.NaT

# Apply the function to the 'mbfc__Date_Established__c' column
accounts['mbfc__Date_Established__c'] = accounts['mbfc__Date_Established__c'].apply(transform_year)


In [161]:
# Format Parent_Parish__c field

# Remove instances of '0'
accounts.Parent_Parish__c = accounts.Parent_Parish__c.str.replace('0', '')



In [162]:
# Append prefix
accounts['Parent_Parish__c'] = accounts['Parent_Parish__c'].apply(lambda x: 'Parishes_' + x if pd.notna(x) and x is not None and x != '' else x)


In [163]:
# Check final results
accounts.Parent_Parish__c[accounts.Parent_Parish__c.isna() == False].sample(10)

40                
59     Parishes_72
144               
101               
163               
106               
87     Parishes_51
46                
47     Parishes_42
127               
Name: Parent_Parish__c, dtype: object

In [164]:
# ParentID field

accounts['ParentId'] = accounts['Parent_Parish__c']


### AccountRecordType & ChurchType


In [165]:
#Sets all rows where AccountRecordType is Church as a Parish.
accounts.loc[accounts['AccountRecordType'] == 'Church', 'mbfc__Church_Type__c'] = 'Parish'
accounts[accounts['AccountRecordType'] == 'Church'].head(5)


,Record Number,AccountRecordType,Formal_Name__c,Name,Parish Name,Archdiocese_Assigns_Clergy__c,Locator_Description__c,BillingCity,BillingState,Mailing Address Province,...,Mailing Address 1,Mailing Address Zip,Vicariate Name,Mailing Address City2,Non_Latin__c,BillingStreet,Religious_Secular_Order__c,Pontifical_or_Diocesan_Order__c,ParentId,mbfc__Church_Type__c
35,1,Church,"Our Lady of Perpetual Help, St Mary’s","Our Lady of Perpetual Help, St Mary’s, Albany",NaN,True,SW Ellsworth St between 8th and 9th Streets,Albany,OR,NaN,...,NaN,NaN,NaN,NaN,False,"Our Lady of Perpetual Help, St Mary’s Parish\n...",None,None,,Parish
36,2,Church,St. Andrew Dũng-Lạc,"St. Andrew Dũng-Lạc Mission, Aloha",NaN,False,SW Grabhorn Rd/209th Ave and Farmington Rd,Aloha,OR,NaN,...,NaN,NaN,NaN,NaN,False,St. Andrew Dũng-Lạc Mission\n7390 SW Grabhorn Rd,None,None,Parishes_83,Parish
37,3,Church,St. Elizabeth Ann Seton,"St. Elizabeth Ann Seton, Aloha",NaN,True,NaN,Aloha,OR,NaN,...,NaN,NaN,NaN,NaN,False,St. Elizabeth Ann Seton Parish\n3145 SW 192nd Ave,None,None,,Parish
38,4,Church,St. Peter the Fisherman,"St. Peter the Fisherman Mission, Arch Cape",NaN,True,79441 Hwy 101 S,Seaside,OR,NaN,...,NaN,NaN,NaN,NaN,False,St. Peter the Fisherman Mission\nPO Box 29,None,None,Parishes_131,Parish
39,5,Church,Our Lady of the Mountain,"Our Lady of the Mountain, Ashland",NaN,True,NaN,Ashland,OR,NaN,...,NaN,NaN,NaN,NaN,False,Our Lady of the Mountain Parish\n987 Hillview Dr,None,None,,Parish


### Generate ExternalId


In [166]:
# Generate an External ID
columns_to_concate = ['src_table', 'Record Number']
accounts = concat_columns(accounts, columns_to_concate, 'Archdpdx_Migration_Id__c', separator='_')

In [167]:
# set Deanery RecordTypeId to the Church RecordTypeId
# map in RecordTypeIds
accounts['RecordTypeId'] = accounts['AccountRecordType'].map(record_types_mapping)
record_types_mapping

{'Church': '012Dx0000003p4xIAA',
 'Deanery': '012Dx0000003p4yIAA',
 'Group': '012Dx0000003p4zIAA',
 'Organization': '012Hu000001pkqEIAQ',
 'Property': '012Dx0000003p51IAA',
 'Religious': '012Dx0000003p5KIAQ',
 'All_Types': '012Dx0000003p53IAA',
 'Any': '012Dx0000003p54IAA',
 'Assignments_Clergy': '012Dx0000003p55IAA',
 'Chancery_Users': '012Dx0000003p56IAA',
 'Clergy_Religious_Residence': '012Dx0000003p57IAA',
 'Diocean_Users': '012Dx0000003p58IAA',
 'Diocesan_Appointment': '012Dx0000003p59IAA',
 'Ecclesial_Affiliation': '012Dx0000003p5AIAQ',
 'Education': '012Dx0000003p5BIAQ',
 'Lay_Person': '012Dx0000003p5HIAQ',
 'Ministerial_Status': '012Dx0000003p5DIAQ',
 'Parish_Affiliations': '012Dx0000003p5EIAQ',
 'Staff': '012Dx0000003p5FIAQ',
 'Consecrated': '012Dx0000003p5GIAQ',
 'Permanent_Deacon': '012Dx0000003p5IIAQ',
 'Priest': '012Dx0000003p5JIAQ',
 'MajorGift': '012Hu000001pkqBIAQ',
 'Grant': '012Hu000001pkqCIAQ',
 'HH_Account': '012Hu000001pkqDIAQ',
 'Donation': '012Hu000001pkqFIAQ',
 

## Load


### Generate a new Job ID


In [168]:
# increment to the job_id
file_name = '/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/jobs/job_id'
curr_job_id = update_job_id(file_name)
print(f"New job ID: {curr_job_id}")

# add/update account DF with job_id
accounts["Job_Id__c"] = curr_job_id


New job ID: 105


### A) Vicariates


In [169]:
# Get Account Group RecordTypeID
deanery_recordTypeId = df_sf_recordTypes.loc[
    (df_sf_recordTypes['DeveloperName'] == 'Deanery') & (df_sf_recordTypes['SobjectType'] == 'Account'),
    'Id'
    ].iloc[0]  # Use .iloc[0] to get the first item if you're expecting exactly one match


# Insert Vicariates holding account
vicariate_account = sf.Account.upsert('Archdpdx_Migration_Id__c/Vicariates_Holding_Acc',
    {
    "Name": "Vicariates",
    "ParentId": diocesan_account_id,
    "mbfc__Diocese__c": diocesan_account_id,
    "RecordTypeId": deanery_recordTypeId,
    # "mbfc__Group_Type__c": 'Office',
    "Job_Id__c": curr_job_id
    }
)

# Get Vicariate Holding Acc's SF ID (as an upsert doesn't return the actual record ID)
vicariate_account = sf.Account.get_by_custom_id('Archdpdx_Migration_Id__c', 'Vicariates_Holding_Acc')
vicariate_account_id = vicariate_account['Id']

vicariate_account_id

'001Dx00001HwDuDIAV'

In [170]:
# Prepare Vicariates staging DF

vicariates = accounts[accounts['AccountRecordType'] == 'Deanery']


vicariates = vicariates[[
    'Record Number',
    'Name',
    # 'AccountRecordType',
    'Job_Id__c',
    'Archdpdx_Migration_Id__c',
    'RecordTypeId'
    ]]

# add parentid
vicariates["mbfc__Diocese__c"] = diocesan_account_id
vicariates['ParentId'] = vicariate_account_id
# vicariates['mbfc__Church_Type__c'] = 'Deanery'
vicariates['RecordTypeId'] = deanery_recordTypeId

vicariates.rename(columns={
        # 'Name, City': 'Name',
        'External_Id': 'Archdpdx_Migration_Id__c'
    }, inplace=True)

vicariates.reset_index()
vicariates.set_index('Record Number', inplace=True)

vicariates

,Name,Job_Id__c,Archdpdx_Migration_Id__c,RecordTypeId,mbfc__Diocese__c,ParentId
Record Number,,,,,,
1,Albany-Corvallis Vicariate,105,Vicariates_1,012Dx0000003p4yIAA,001Dx00001HwDsgIAF,001Dx00001HwDuDIAV
2,"Beaverton, Suburban Vicariate",105,Vicariates_2,012Dx0000003p4yIAA,001Dx00001HwDsgIAF,001Dx00001HwDuDIAV
3,Columbia County Vicariate,105,Vicariates_3,012Dx0000003p4yIAA,001Dx00001HwDsgIAF,001Dx00001HwDuDIAV
4,Downtown Portland Vicariate,105,Vicariates_4,012Dx0000003p4yIAA,001Dx00001HwDsgIAF,001Dx00001HwDuDIAV
5,"East Portland, Suburban Vicariate",105,Vicariates_5,012Dx0000003p4yIAA,001Dx00001HwDsgIAF,001Dx00001HwDuDIAV
6,Marion County Vicariate,105,Vicariates_6,012Dx0000003p4yIAA,001Dx00001HwDsgIAF,001Dx00001HwDuDIAV
7,Metropolitan Eugene Vicariate,105,Vicariates_7,012Dx0000003p4yIAA,001Dx00001HwDsgIAF,001Dx00001HwDuDIAV
8,Metropolitan Salem Vicariate,105,Vicariates_8,012Dx0000003p4yIAA,001Dx00001HwDsgIAF,001Dx00001HwDuDIAV
9,North Coast Vicariate,105,Vicariates_9,012Dx0000003p4yIAA,001Dx00001HwDsgIAF,001Dx00001HwDuDIAV


#### Export Vicariates to CSV


In [171]:
# export to CSV
vicariates.to_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/vicariates_staging.csv')


#### Upsert Vicariates


In [172]:
bulk_data = []
for row in vicariates.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

if run_upserts == 'True':
    vicariate_upsert = sf.bulk.Account.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    upserts = pd.DataFrame(vicariate_upsert)

    print(upserts)
    

    success  created                  id errors
0      True    False  001Dx00001HwDwnIAF     []
1      True    False  001Dx00001HwDwoIAF     []
2      True    False  001Dx00001HwDwpIAF     []
3      True    False  001Dx00001HwDwqIAF     []
4      True    False  001Dx00001HwDwrIAF     []
5      True    False  001Dx00001HwDwsIAF     []
6      True    False  001Dx00001HwDwtIAF     []
7      True    False  001Dx00001HwDwuIAF     []
8      True    False  001Dx00001HwDwvIAF     []
9      True    False  001Dx00001HwDwwIAF     []
10     True    False  001Dx00001HwDwxIAF     []
11     True    False  001Dx00001HwDwyIAF     []
12     True    False  001Dx00001HwDwzIAF     []
13     True    False  001Dx00001HwDx0IAF     []
14     True    False  001Dx00001HwDx1IAF     []
15     True    False  001Dx00001HwDx2IAF     []
16     True    False  001Dx00001HwDx3IAF     []
17     True    False  001Dx00001HwDx4IAF     []


In [173]:
# Generate an Errors log
import csv

keys = vicariate_upsert[0].keys()

with open('results_files/vicariate_results', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, keys)
    writer.writeheader()
    writer.writerows(vicariate_upsert)

In [174]:
# Get Vicariate records from SF

sf_deaneries = sf.query("SELECT Archdpdx_Migration_Id__c, Id FROM Account WHERE RecordType.DeveloperName = 'Deanery'")

df_sf_deaneries = pd.DataFrame(sf_deaneries['records'])
df_sf_deaneries = df_sf_deaneries.drop(columns = 'attributes')

df_sf_deaneries

# Creates a dict of Vicariate unique ids to the new Salesforce record IDs, so can populate on latter Account records
vicariate_sf_recordids = df_sf_deaneries.set_index('Archdpdx_Migration_Id__c')['Id'].to_dict()
vicariate_sf_recordids

{'Vicariates_Holding_Acc': '001Dx00001HwDuDIAV',
 'Vicariates_1': '001Dx00001HwDwnIAF',
 'Vicariates_2': '001Dx00001HwDwoIAF',
 'Vicariates_3': '001Dx00001HwDwpIAF',
 'Vicariates_4': '001Dx00001HwDwqIAF',
 'Vicariates_5': '001Dx00001HwDwrIAF',
 'Vicariates_6': '001Dx00001HwDwsIAF',
 'Vicariates_7': '001Dx00001HwDwtIAF',
 'Vicariates_8': '001Dx00001HwDwuIAF',
 'Vicariates_9': '001Dx00001HwDwvIAF',
 'Vicariates_10': '001Dx00001HwDwwIAF',
 'Vicariates_11': '001Dx00001HwDwxIAF',
 'Vicariates_12': '001Dx00001HwDwyIAF',
 'Vicariates_13': '001Dx00001HwDwzIAF',
 'Vicariates_14': '001Dx00001HwDx0IAF',
 'Vicariates_15': '001Dx00001HwDx1IAF',
 'Vicariates_16': '001Dx00001HwDx2IAF',
 'Vicariates_17': '001Dx00001HwDx3IAF',
 'Vicariates_18': '001Dx00001HwDx4IAF'}

### B) Parishes, Schools, Organizations


In [175]:
# Create acc_main (accounts excluding Deaneries (already handled) and Religious (to be handled differently, after))
acc_main = accounts[accounts['AccountRecordType'] != 'Deanery']
acc_main = acc_main[acc_main['AccountRecordType'] != 'Religious']

acc_main.loc[acc_main['AccountRecordType'] == 'Church', 'Vicariate_Ext_Id'] = 'Vicariates_' + acc_main['Vicariate']

In [176]:
acc_main.sample(5)

,Record Number,AccountRecordType,Formal_Name__c,Name,Parish Name,Archdiocese_Assigns_Clergy__c,Locator_Description__c,BillingCity,BillingState,Mailing Address Province,...,Non_Latin__c,BillingStreet,Religious_Secular_Order__c,Pontifical_or_Diocesan_Order__c,ParentId,mbfc__Church_Type__c,Archdpdx_Migration_Id__c,RecordTypeId,Job_Id__c,Vicariate_Ext_Id
106,76,Church,Ascension,"Ascension, Portland",NaN,True,SE Alder St and 76th Ave,Portland,OR,NaN,...,False,Ascension Parish\n743 SE 76th Ave,None,None,,Parish,Parishes_76,012Dx0000003p4xIAA,105,Vicariates_5
122,93,Church,St. Clare,"St. Clare, Portland",NaN,True,SW 19th Ave and Spring Garden St,Portland,OR,NaN,...,False,St. Clare Parish\n8535 SW 19th Ave,None,None,,Parish,Parishes_93,012Dx0000003p4xIAA,105,Vicariates_17
27,53,Organization,Providence Medford Medical Center,Providence Medford Medical Center,NaN,False,NaN,Medford,OR,NaN,...,False,1111 Crater Lake Ave,None,None,NaN,NaN,Offices_53,012Hu000001pkqEIAQ,105,NaN
42,9,Church,Holy Trinity,"Holy Trinity, Beaverton",NaN,True,NaN,Beaverton,OR,NaN,...,False,Holy Trinity Parish\n13715 SW Walker Rd,None,None,,Parish,Parishes_9,012Dx0000003p4xIAA,105,Vicariates_2
73,43,Church,St. Anne,"St. Anne, Gresham",NaN,True,NaN,Portland,OR,NaN,...,False,St. Anne Parish\n1015 SE 182nd Ave,None,None,,Parish,Parishes_43,012Dx0000003p4xIAA,105,Vicariates_5


In [177]:
# map in Deaneries
acc_main['mbfc__Deanery__c'] = acc_main.Vicariate_Ext_Id.map(vicariate_sf_recordids)

acc_main[acc_main['AccountRecordType'] == 'Church']['mbfc__Deanery__c']

35     001Dx00001HwDwnIAF
36     001Dx00001HwDwzIAF
37     001Dx00001HwDx2IAF
38     001Dx00001HwDwvIAF
39     001Dx00001HwDx1IAF
              ...        
181    001Dx00001HwDwrIAF
182    001Dx00001HwDx3IAF
183    001Dx00001HwDwsIAF
184    001Dx00001HwDx4IAF
185    001Dx00001HwDwtIAF
Name: mbfc__Deanery__c, Length: 151, dtype: object

In [178]:
# Clean up NaN values

acc_main.fillna('', inplace=True)

In [179]:
# Generate Schedule text from all Schedule columns

def create_account_schedule(row):
    account_schedule = []
    for i in range(1, 8):
        head_col = f'Schedule {i} Head'
        text_col = f'Schedule {i} Text'
        
        head = row[head_col]
        text = row[text_col]
        
        if pd.notnull(head) or pd.notnull(text):
            if pd.notnull(head):
                account_schedule.append(f"<p><strong>{head}</strong></p>")
            if pd.notnull(text):
                account_schedule.append(f"<p>{text}</p>")
            account_schedule.append("<p><br></p>")
    
    # Join all parts into a single string
    return "".join(account_schedule).strip()

acc_main['Account_Schedule__c'] = acc_main.apply(create_account_schedule, axis=1)



In [180]:
acc_main['Account_Schedule__c'].sample(15)

290    <p><strong></strong></p><p></p><p><br></p><p><...
310    <p><strong></strong></p><p></p><p><br></p><p><...
94     <p><strong>Weekend Mass</strong></p><p>8:30 AM...
19     <p><strong></strong></p><p></p><p><br></p><p><...
272    <p><strong></strong></p><p></p><p><br></p><p><...
98     <p><strong>Weekend Mass</strong></p><p>Saturda...
135    <p><strong>Weekend Mass</strong></p><p>Saturda...
81     <p><strong>Weekend Mass</strong></p><p>Sunday ...
10     <p><strong></strong></p><p></p><p><br></p><p><...
180    <p><strong>Weekend Mass</strong></p><p>Saturda...
306    <p><strong></strong></p><p></p><p><br></p><p><...
279    <p><strong></strong></p><p></p><p><br></p><p><...
119    <p><strong>Weekend Mass</strong></p><p>Saturda...
144    <p><strong>Weekend Mass</strong></p><p>Saturda...
86     <p><strong>Weekend Mass</strong></p><p>Saturda...
Name: Account_Schedule__c, dtype: object

In [181]:
acc_main

,Record Number,AccountRecordType,Formal_Name__c,Name,Parish Name,Archdiocese_Assigns_Clergy__c,Locator_Description__c,BillingCity,BillingState,Mailing Address Province,...,Religious_Secular_Order__c,Pontifical_or_Diocesan_Order__c,ParentId,mbfc__Church_Type__c,Archdpdx_Migration_Id__c,RecordTypeId,Job_Id__c,Vicariate_Ext_Id,mbfc__Deanery__c,Account_Schedule__c
0,1,Organization,Pastoral Center,Pastoral Center,,True,,Portland,OR,,...,,,,,Offices_1,012Hu000001pkqEIAQ,105,,,<p><strong></strong></p><p></p><p><br></p><p><...
1,3,Organization,Catholic Sentinel,Catholic Sentinel,,False,,Portland,OR,,...,,,,,Offices_3,012Hu000001pkqEIAQ,105,,,<p><strong></strong></p><p></p><p><br></p><p><...
2,4,Organization,Catholic Cemeteries,Catholic Cemeteries,,False,,Portland,OR,,...,,,,,Offices_4,012Hu000001pkqEIAQ,105,,,<p><strong></strong></p><p></p><p><br></p><p><...
3,6,Organization,Griffin Center,Griffin Center,,False,,Milwaukie,OR,,...,,,,,Offices_6,012Hu000001pkqEIAQ,105,,,<p><strong></strong></p><p></p><p><br></p><p><...
4,11,Organization,Providence Portland Medical Center,Providence Portland Medical Center,,True,,Portland,OR,,...,,,,,Offices_11,012Hu000001pkqEIAQ,105,,,<p><strong></strong></p><p></p><p><br></p><p><...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,58,Organization,Resurrection Catholic Parish School,"Resurrection Catholic Parish School, Tualatin",,True,,Tualatin,OR,,...,,,,,Schools_58,012Hu000001pkqEIAQ,105,,,<p><strong></strong></p><p></p><p><br></p><p><...
330,1,Organization,OSU Newman Center,"OSU Newman Center, Corvallis",,False,,Corvallis,OR,,...,,,,,NewmanCenters_1,012Hu000001pkqEIAQ,105,,,<p><strong>Mass (During Academic Year)</strong...
331,2,Organization,St. Thomas More (UO) Newman Center,"St. Thomas More (UO) Newman Center, Eugene",,False,,Eugene,OR,,...,,,,,NewmanCenters_2,012Hu000001pkqEIAQ,105,,,<p><strong>Weekend Mass</strong></p><p>Saturda...
332,3,Organization,Walsh Memorial (SOU) Newman Center at Our Lady...,Walsh Memorial (SOU) Newman Center at Our Lady...,,True,,Ashland,OR,,...,,,,,NewmanCenters_3,012Hu000001pkqEIAQ,105,,,<p><strong>Sunday Mass</strong></p><p>5:00 pm ...


In [182]:
# Create 'account_staging' df (drop extraneous columns)

accounts_staging = acc_main[[
    'Name',
    'Formal_Name__c',
    'RecordTypeId',
    'mbfc__Church_Type__c',
    'mbfc__Deanery__c',
    'BillingStreet',
    'BillingCity',
    'BillingState',
    'BillingPostalCode',
    'BillingCountry',
    'Phone',
    'Fax',
    'mbfc__Email__c',
    'Website',
    'Account_Schedule__c',
    'mbfc__Abbreviation__c',
    'mbfc__Religious_Suffix__c',
    'mbfc__Type_Members__c',
    'Description',
    'Archdiocese_Assigns_Clergy__c', # Boolean fields
    'Non_Latin__c', 
    'Disabled_Access__c', 
    'Locator_Description__c',
    'Parent_Parish__c',
    'mbfc__Date_Established__c',
    'County__c',
    'Sanctuary_Capacity__c',
    # 'Miles_to_Pastoral_Centre__c',
    'Religious_Secular_Order__c',
    'Pontifical_or_Diocesan_Order__c',
    'Archdiocesan_School_Code__c',
    'Grades_Provided__c',
    'Job_Id__c',
    'Archdpdx_Migration_Id__c',
    # 'ParentId'  # Later, check whether or not can upsert using external ID using this field

    ]]

In [183]:
accounts_staging

,Name,Formal_Name__c,RecordTypeId,mbfc__Church_Type__c,mbfc__Deanery__c,BillingStreet,BillingCity,BillingState,BillingPostalCode,BillingCountry,...,Parent_Parish__c,mbfc__Date_Established__c,County__c,Sanctuary_Capacity__c,Religious_Secular_Order__c,Pontifical_or_Diocesan_Order__c,Archdiocesan_School_Code__c,Grades_Provided__c,Job_Id__c,Archdpdx_Migration_Id__c
0,Pastoral Center,Pastoral Center,012Hu000001pkqEIAQ,,,2838 E Burnside St,Portland,OR,97214,,...,,NaT,,,,,,,105,Offices_1
1,Catholic Sentinel,Catholic Sentinel,012Hu000001pkqEIAQ,,,2838 E Burnside St,Portland,OR,97214,,...,,NaT,,,,,,,105,Offices_3
2,Catholic Cemeteries,Catholic Cemeteries,012Hu000001pkqEIAQ,,,333 SW Skyline Blvd,Portland,OR,97221,,...,,NaT,,,,,,,105,Offices_4
3,Griffin Center,Griffin Center,012Hu000001pkqEIAQ,,,11957 SE Fuller Rd,Milwaukie,OR,97222,,...,,NaT,,,,,,,105,Offices_6
4,Providence Portland Medical Center,Providence Portland Medical Center,012Hu000001pkqEIAQ,,,4805 NE Glisan St,Portland,OR,97213,,...,,NaT,,,,,,,105,Offices_11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,"Resurrection Catholic Parish School, Tualatin",Resurrection Catholic Parish School,012Hu000001pkqEIAQ,,,,Tualatin,OR,,,...,,NaT,,,,,12-WEESRES,PK-5,105,Schools_58
330,"OSU Newman Center, Corvallis",OSU Newman Center,012Hu000001pkqEIAQ,,,2127 NW Monroe Ave,Corvallis,OR,97330,,...,,NaT,,,,,,,105,NewmanCenters_1
331,"St. Thomas More (UO) Newman Center, Eugene",St. Thomas More (UO) Newman Center,012Hu000001pkqEIAQ,,,1850 Emerald St,Eugene,OR,97403,,...,,1915-01-01,,,,,,,105,NewmanCenters_2
332,Walsh Memorial (SOU) Newman Center at Our Lady...,Walsh Memorial (SOU) Newman Center at Our Lady...,012Hu000001pkqEIAQ,,,987 Hillview Dr,Ashland,OR,97520,,...,,NaT,,,,,,,105,NewmanCenters_3


#### Create Parishes Holding Acc for acc heirarchy

In [184]:
# Upsert a Parishes holding account

# Get Account Group RecordTypeID
group_recordTypeId = df_sf_recordTypes.loc[
    (df_sf_recordTypes['DeveloperName'] == 'Group') & (df_sf_recordTypes['SobjectType'] == 'Account'),
    'Id'
    ].iloc[0]  # Use .iloc[0] to get the first item if you're expecting exactly one match


# Insert Vicariates holding account
parish_holding_account = sf.Account.upsert('Archdpdx_Migration_Id__c/Parishes_Holding_Acc',
    {
    "Name": "Parishes",
    "ParentId": diocesan_account_id,
    "RecordTypeId": group_recordTypeId,
    "Job_Id__c": curr_job_id,
    "mbfc__Group_Type__c": "Office"
    }
)

# Get Vicariate Holding Acc's SF ID (as an upsert doesn't return the actual record ID)

parish_holding_account = sf.Account.get_by_custom_id('Archdpdx_Migration_Id__c', 'Parishes_Holding_Acc')

parishes_holding_account_id = parish_holding_account['Id']

parishes_holding_account_id

'001Dx00001HwDxKIAV'

In [185]:
# Set the ParentId for all Parish records

accounts_staging['ParentId'] = None

accounts_staging['ParentId']= accounts_staging.apply(
    lambda row: parishes_holding_account_id if row['mbfc__Church_Type__c'] == 'Parish' else row['ParentId'], axis=1
)

accounts_staging.sample(10)


/var/folders/_g/p51k4k2d5fg45slnqjhgd3sc0000gn/T/ipykernel_45128/2616635530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accounts_staging['ParentId'] = None
/var/folders/_g/p51k4k2d5fg45slnqjhgd3sc0000gn/T/ipykernel_45128/2616635530.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accounts_staging['ParentId']= accounts_staging.apply(


,Name,Formal_Name__c,RecordTypeId,mbfc__Church_Type__c,mbfc__Deanery__c,BillingStreet,BillingCity,BillingState,BillingPostalCode,BillingCountry,...,mbfc__Date_Established__c,County__c,Sanctuary_Capacity__c,Religious_Secular_Order__c,Pontifical_or_Diocesan_Order__c,Archdiocesan_School_Code__c,Grades_Provided__c,Job_Id__c,Archdpdx_Migration_Id__c,ParentId
92,"St. John the Baptist, Milwaukie",St. John the Baptist,012Dx0000003p4xIAA,Parish,001Dx00001HwDwzIAF,St. John the Baptist Parish\n10955 SE 25th Ave,Milwaukie,OR,97222,,...,1864-01-01,Clackamas,900.0,,,,,105,Parishes_62,001Dx00001HwDxKIAV
278,"St. Matthew Catholic School, Hillsboro",St. Matthew Catholic School,012Hu000001pkqEIAQ,,,,Hillsboro,OR,,,...,1914-01-01,,,,,12-HILMATS,PS-8,105,Schools_25,None
183,"St. Luke, Woodburn",St. Luke,012Dx0000003p4xIAA,Parish,001Dx00001HwDwsIAF,St. Luke Parish\n417 Harrison St,Woodburn,OR,97071,,...,1899-01-01,Marion,700.0,,,,,105,Parishes_154,001Dx00001HwDxKIAV
147,"St. Mary’s-by-the-Sea, Rockaway",St. Mary’s-by-the-Sea,012Dx0000003p4xIAA,Parish,001Dx00001HwDwvIAF,St. Mary’s-by-the-Sea Parish\nPO Box 390,Rockaway,OR,97136,,...,1927-01-01,Tillamook,210.0,,,,,105,Parishes_118,001Dx00001HwDxKIAV
263,"Valley Catholic High School, Beaverton",Valley Catholic High School,012Hu000001pkqEIAQ,,,,Beaverton,OR,,,...,NaT,,,,,12-BEAVALH,9-12,105,Schools_10,None
294,"The Madeleine School, Portland",The Madeleine School,012Hu000001pkqEIAQ,,,,Portland,OR,,,...,1912-01-01,,,,,12-PDXMADS,K-8,105,Schools_41,None
185,"St. Thomas More Newman Center, Eugene",St. Thomas More Newman Center,012Dx0000003p4xIAA,Parish,001Dx00001HwDwtIAF,St. Thomas More Newman Center Parish\n1850 Eme...,Eugene,OR,97403,,...,1967-01-01,Lane,0.0,,,,,105,Parishes_173,001Dx00001HwDxKIAV
90,"St. Catherine of Siena Mission, Mill City",St. Catherine of Siena,012Dx0000003p4xIAA,Parish,001Dx00001HwDwxIAF,St. Catherine of Siena Mission\n1077 N 6th Ave,Stayton,OR,97383,,...,NaT,Linn,0.0,,,,,105,Parishes_60,001Dx00001HwDxKIAV
180,"St. Anthony, Waldport",St. Anthony,012Dx0000003p4xIAA,Parish,001Dx00001HwDwnIAF,St. Anthony Parish\nPO Box 770,Waldport,OR,97394,,...,1949-01-01,Lincoln,0.0,,,,,105,Parishes_151,001Dx00001HwDxKIAV
256,"St. Rose School, Portland",St. Rose School,012Hu000001pkqEIAQ,,,,Portland,OR,,,...,1913-01-01,,,,,12-PDXARCS,PK-8,105,Schools_1,None


#### Upsert Accounts (TBD )


In [186]:
# send accounts_staging to csv
accounts_staging.to_csv('staging_files/accounts_staging.csv', encoding='utf-8-sig')

In [187]:
# FIXME: Format ExternalID lookups into dictionary to match SF's api so can upsert using simple-salesforce

# Rename columns apis
accounts_staging = accounts_staging.rename(columns={'Parent_Parish__c': 'Parent_Parish__r'})  # Later on, attempt to include 'ParentId' (which, as a standard SF field, might not work)

# Reformat values to match what SF api requires
accounts_staging['Parent_Parish__r'] = accounts_staging.apply(lambda x: "{'Archdpdx_Migration_Id__c': '" + x['Parent_Parish__r'] + "'}" if pd.notna(x['Parent_Parish__r']) and x['Parent_Parish__r'] != 'None' and x['Parent_Parish__r'] != '' else None, axis=1)




In [192]:
# Attempt to upsert using new function
# FIXME: This upsert isn't working but appears to have worked previously (according to the 'accounts_results' file)

accounts_upsert2 = upsert_to_salesforce_bulk(sf, accounts_staging, 'Account', 'Archdpdx_Migration_Id__c', 'results_files/accounts_failed', batch_size=10000)

Bulk upsert completed. Successful upserts: 0, Failed upserts: 247


In [ ]:
# Extract SF Account records

sf_accounts = sf.query('Select id, Name, RecordTypeId, mbfc__Church_Type__c, Archdpdx_Migration_Id__c, Job_Id__c from Account WHERE Job_Id__c != null')
sf_accounts = pd.DataFrame(sf_accounts['records'])
sf_accounts = sf_accounts.drop(columns = 'attributes')
sf_accounts

### C) Religious Institutes (Parents)


In [ ]:
"""
- 'acc_religious' DF: create unique_id of religious parents
- create 'acc_religious_orders' DF , upsert into SF
- extract accounts from Salesforce, create dict (external_ID : account_ID)
- map parent ids onto religious child accounts DF in main DF
- 'acc_religious' > staging DF ('acc_religious')
    - drop unnecessary columns
    - upsert create DF of religious children, upsert into SF with
"""

# Create a new DF of all Religious accounts
acc_religious = accounts[accounts['AccountRecordType'] == 'Religious']

# Create a simplified external ID field
acc_religious['Archdpdx_Migration_Id__c'] = acc_religious['Order Full Name'].apply(
    lambda x: x.lower().replace(' ', '')[:40]
)

acc_religious_2 = acc_religious

# Create a DF for only parent religious order accounts
acc_religious_parents = acc_religious_2[[
    'Order Full Name', 
    'Name', 
    'mbfc__Abbreviation__c', 
    'mbfc__Religious_Suffix__c', 
    'mbfc__Type_Members__c', 
    'Archdpdx_Migration_Id__c',
    'Pontifical_or_Diocesan_Order__c',
    'Religious_Secular_Order__c',
    ]]

# Drop duplicate rows of the same parent Religious Order (becuase there are more than 1 local community of a particular order)
acc_religious_parents.drop_duplicates('Order Full Name', inplace=True)

# How many remaining rows after dropping duplicates?
print(acc_religious_parents.shape)

# Rename columns
acc_religious_parents = acc_religious_parents.rename(columns={
    'Order Full Name': 'Description'
    })

# Drop NA
acc_religious_parents.fillna('', inplace=True)

acc_religious_parents


In [ ]:
acc_religious_parents['mbfc__Religious_Type__c'] = 'Congregation'

In [ ]:
# Set recordType to 'Religious'

religious_recordtype_id = df_sf_recordTypes.loc[
    (df_sf_recordTypes['DeveloperName'] == 'Religious') & (df_sf_recordTypes['SobjectType'] == 'Account'),
    'Id'
    ].iloc[0]  # Use .iloc[0] to get the first item if you're expecting exactly one match

print(religious_recordtype_id)

acc_religious_parents['RecordTypeId'] = religious_recordtype_id

acc_religious_parents.sample(10)

In [ ]:
# Send to CSV
acc_religious_parents.to_csv('staging_files/religious_order_staging.csv', encoding='utf-8-sig')

In [ ]:
# Upsert to Salesforce
bulk_data = []
for row in acc_religious_parents.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

if run_upserts == 'True':
    religious_order_upsert = sf.bulk.Account.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    df_rel_order_upsert = pd.DataFrame(religious_order_upsert)

df_rel_order_upsert

In [ ]:
# Generate an Errors log
import csv

keys = religious_order_upsert[0].keys()

with open('results_files/religious_order_results', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, keys)
    writer.writeheader()
    writer.writerows(religious_order_upsert)

In [ ]:
# @title get SF Accounts
get_all_rel_accounts = f"Select id, Name, RecordTypeId, Type, Archdpdx_Migration_Id__c from Account where RecordTypeID = '{religious_recordtype_id}'"

print(religious_recordtype_id)

# get list of records, add to dataframe
sf_accounts = sf.query(get_all_rel_accounts)
df_sf_accounts = pd.DataFrame(sf_accounts['records'])
df_sf_accounts = df_sf_accounts.drop(columns = 'attributes')

df_sf_accounts.sample(10)

In [ ]:
religious_order_mapping = df_sf_accounts.set_index('Archdpdx_Migration_Id__c')['Id'].to_dict()
# religious_order_mapping

### D) Religious Communities


In [ ]:
acc_religious_staging = (acc_religious
                         .rename(columns={'Archdpdx_Migration_Id__c' : 'Parent_Archdpdx_Migration_Id__c'})
)

acc_religious_staging['ParentId'] = acc_religious_staging['Parent_Archdpdx_Migration_Id__c'].map(religious_order_mapping)

In [ ]:
# Enrich the data

acc_religious_staging['mbfc__Religious_Type__c'] = 'Local Community'
acc_religious_staging['Archdpdx_Migration_Id__c'] = 'RelCommunities_' + acc_religious_staging['Record Number'].astype('str')
acc_religious_staging['RecordTypeId'] = religious_recordtype_id
acc_religious_staging.drop(columns='Name', inplace=True)
acc_religious_staging.rename(columns={
    'Name, City': 'Name'
}, inplace=True)

acc_religious_staging.sample(5)

In [ ]:
acc_religious_staging_2 = acc_religious_staging[[
    'Name',
    'RecordTypeId',
    'mbfc__Religious_Type__c',
    'BillingStreet',
    'BillingCity',
    'BillingState',
    'BillingPostalCode',
    'BillingCountry',
    'Phone',
    'Fax',
    'mbfc__Email__c',
    'Website',
    'mbfc__Abbreviation__c',
    'mbfc__Religious_Suffix__c',
    'mbfc__Type_Members__c',
    'Description',
    'Job_Id__c',
    'ParentId',
    'Archdpdx_Migration_Id__c'
    ]]

acc_religious_staging_2.sample(5)

In [ ]:
# Final Cleanup

acc_religious_staging_2 = acc_religious_staging_2.fillna('')

In [ ]:
# @title Send to CSV
acc_religious_staging_2.to_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/religious_community_staging.csv', encoding='utf-8-sig')

In [ ]:
# @title Upsert to Salesforce
bulk_data = []
for row in acc_religious_staging_2.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

if run_upserts == 'True':
    religious_community_upsert = sf.bulk.Account.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    df_rel_community_upsert = pd.DataFrame(religious_community_upsert)

df_rel_community_upsert

### E) Religious Superiors


In [ ]:
acc_rel_superiors = acc_religious_2[[
    'Name',
    'Major Superior Name',
    'Major Superior Phone',
    'Major Superior Email',
    'Archdpdx_Migration_Id__c']]


acc_rel_superiors['AccountId'] = acc_rel_superiors.Archdpdx_Migration_Id__c.map(religious_order_mapping)

# acc_rel_superiors.sample(5)

In [ ]:
# @title Parse Complex Names
def parse_names(df, column_name):
    # Convert all non-string entries to strings (handling NaN and other data types)
    df[column_name] = df[column_name].fillna('').apply(str)

    # Create a new DataFrame to store the name parts
    name_parts = pd.DataFrame()

    # Parse each name in the column
    name_parts['First Name'] = df[column_name].apply(lambda x: HumanName(x).first if x.strip() != '' else '')
    name_parts['Last Name'] = df[column_name].apply(lambda x: HumanName(x).last if x.strip() != '' else '')
    name_parts['Middle Name'] = df[column_name].apply(lambda x: HumanName(x).middle if x.strip() != '' else '')
    name_parts['Title'] = df[column_name].apply(lambda x: HumanName(x).title if x.strip() != '' else '')
    name_parts['Suffix'] = df[column_name].apply(lambda x: HumanName(x).suffix if x.strip() != '' else '')
    name_parts['Nickname'] = df[column_name].apply(lambda x: HumanName(x).nickname if x.strip() != '' else '')

    # Combine the original DataFrame with the name parts DataFrame
    result_df = pd.concat([df, name_parts], axis=1)
    return result_df



In [ ]:
!pip install nameparser
from nameparser import HumanName
from nameparser.config import CONSTANTS

# Add dataset-specific Titles and Suffix constants for parsing
CONSTANTS.titles.add('Very', 'Rev.', 'Very Rev.', 'Sr.')
CONSTANTS.suffix_acronyms.add('FRS', 'OMI', 'OSA', 'OCD', 'OP', 'OC', 'FSE', 'OMV', 'SDB', 'SM', 'SFX', 'SP', 'OP', 'O.S.M', 'SNJM', 'OSF', 'HMRF', 'DD', 'CSJP', 'SDD', 'BVM', 'BVM - President' )


In [ ]:
# Parse Complex Names
acc_rel_superiors_parsed = parse_names(acc_rel_superiors, 'Major Superior Name')

In [ ]:
# @title Final cleanup

acc_rel_superiors_staging = acc_rel_superiors_parsed.fillna('')

acc_rel_superiors_staging['Archdpdx_Migration_Id__c'] = acc_rel_superiors_staging['Major Superior Name'].apply(lambda x: x.replace(' ','').lower())

# Rename columns
acc_rel_superiors_staging = acc_rel_superiors_staging.rename(columns={
    'Major Superior Phone': 'Phone',
    'Major Superior Email': 'Email',
    'Title': 'Salutation',
    'First Name': 'FirstName',
    'Middle Name': 'MiddleName',
    'Last Name': 'LastName'
})

# Add job id
acc_rel_superiors_staging['Archdpdx_Job_Id__c'] = curr_job_id

# Drop columns
acc_rel_superiors_staging = acc_rel_superiors_staging.drop(columns=['Name', 'Major Superior Name', 'Nickname'])

# Drop empty rows
acc_rel_superiors_staging = acc_rel_superiors_staging[acc_rel_superiors_staging['LastName'].str.strip() != '']

acc_rel_superiors_staging.sample(10)

In [ ]:
# @title Send to CSV
acc_rel_superiors_staging.to_csv('staging_files/religious_superiors_staging.csv', encoding='utf-8-sig')

In [ ]:
# Upsert to Salesforce

def find_existing_contact(sf, first_name, last_name):
    query = f"SELECT Id, Archdpdx_Migration_Id__c FROM Contact WHERE FirstName = '{first_name}' AND LastName = '{last_name}'"
    result = sf.query(query)
    return result['records']



bulk_data = []
for row in acc_rel_superiors_staging.itertuples(index=False):
    d = row._asdict()
    existing_contacts = find_existing_contact(sf, d['FirstName'], d['LastName'])
    if existing_contacts:
        # Update existing contact with external ID
        d['Id'] = existing_contacts[0]['Id']
        bulk_data.append(d)
    else:
        bulk_data.append(d)


if run_upserts == 'True':
    religious_superior_upsert = sf.bulk.Contact.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    df_rel_superior_upsert = pd.DataFrame(religious_superior_upsert)

df_rel_superior_upsert

In [ ]:
# Update Religious Communities with Rel. Superior

# TODO: It would take much less time to simply do this post-migration manually.

# CONTACTS


## Extract


In [ ]:
import pandas as pd
df_contacts = (pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/People.csv')
               .set_index('Record Number', verify_integrity=True)
               .drop(index='recNum') # Drops the extra row that replicates the labels
               .rename(columns=lambda x: x.replace(' ', '_')) # Remove whitespace in column names
)

df_contacts.sample(10)


#### Get Photos


In [ ]:
import os
import pandas as pd

# def list_jpeg_files(directory):
#     data = []
#     for filename in os.listdir(directory):
#         if filename.endswith(".jpeg") or filename.endswith(".jpg"):  # Checking for jpeg files
#             full_path = os.path.join(directory, filename)
#             data.append({'Filename': filename, 'Full Path': full_path})
#     return pd.DataFrame(data)

# # Specify your directory
# directory = '/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/sql_backup/archdpdx.info backups/public_html/people/graphics/portraits/large'
# jpeg_files_df = list_jpeg_files(directory)


In [ ]:
# # Query for the Library
# library_query = "SELECT Id, Name FROM ContentWorkspace WHERE Name = 'ADPDX Person Profile Photos'"
# library_result = sf.query(library_query)

# # Check if the library exists and get its ID
# if library_result['records']:
#     library_id = library_result['records'][0]['Id']
#     print(f"Library ID: {library_id}")

#     # Query for the Folder within the Library
#     folder_query = f"SELECT Id, Name FROM ContentFolder WHERE ParentContentFolderId = '{library_id}'"
#     folder_result = sf.query(folder_query)

#     # Check if the folder exists and get its ID
#     if folder_result['records']:
#         folder_id = folder_result['records'][0]['Id']
#         print(f"Folder ID: {folder_id}")
#     else:
#         print("Folder 'Large JPEGs' not found in the library.")
# else:
#     print("Library 'ADPDX Person Profile Photos' not found.")

## Analysis

Here we check the various columns and their types, count where values exist, count of unique values, sample data, etc.

DF shape:

- 142 columns
- 3017 rows


In [ ]:
# Check the original shape of the imported CSV
print(f"Shape of original data set: {df_contacts.shape}")

# export to csv a list of the contact fields with count, unique, top, freq
contacts_describe = df_contacts.describe(include='all').transpose()
contacts_describe.to_csv(f'/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/analysis/contacts_describe.csv')

df_contacts.describe(include='all').transpose()  #initial analysis of the Contacts table

In [ ]:
unique_languages = df_contacts['Languages'].unique()
unique_languages

In [ ]:
# import re
# import numpy as np


# def deduplicate_languages(list_languages):
#     # Define a regular expression pattern to match periods and punctuation
#     punctuation_pattern = r'[.,!?;:"]'

#     # Flatten the array and filter out NaN values
#     flattened_languages = [re.sub(punctuation_pattern, '', lang) for sublist in list_languages if pd.notna(sublist) for lang in sublist.split(',')]

#     # Deduplicate the list of languages
#     unique_languages = list(set(flattened_languages))

#     return unique_languages


# # Example usage:
# unique_languages = deduplicate_languages(unique_languages)
# print(unique_languages)


## Transform


In [ ]:
# list of columns NOT to be migrated as Contact attributes
misc_columns_to_drop = [
    'Password',
    'Password_Must_be_Changed',
    'Sort_Name'
]

affiliation_columns = [
    'Baptism_Date',
    'Place_of_Baptism',
    'Confirmation_Date',
    'Place_of_Confirmation',
    'Received_Date',
    'Parish_of_Record',
    'Marriage_Date',
    'Place_of_Marriage',
    'Date_of_First_Vows',
    'Date_of_Final_Vows',
    'Reader_Date',
    'Acolyte_Date',
    'Bachelor_Degree_Year',
    'Bachelor_Degree_Type',
    'Bachelor_Degree_Institution',
    'Graduate_1_Degree_Institution',
    'Graduate_1_Degree_Type',
    'Graduate_1_Degree_Year',
    'Graduate_2_Degree_Institution',
    'Graduate_2_Degree_Type',
    'Graduate_2_Degree_Year',
    'Graduate_3_Degree_Institution',
    'Graduate_3_Degree_Type',
    'Graduate_3_Degree_Year',
    'Graduate_4_Degree_Institution',
    'Graduate_4_Degree_Type',
    'Graduate_4_Degree_Year',
    'Diaconal_Ordination_Date',
    'Diaconal_Ordination_Place',
    'Diaconal_Ordination_Prelate',
    'Presbyteral_Ordination_Date',
    'Presbyteral_Ordination_Place',
    'Presbyteral_Ordination_Prelate',
    'Episcopal_Ordination_Date',
    'Episcopal_Ordination_Place',
    'Episcopal_Ordination_Prelate',
    'Incardinated_From_Date',
    'Incardinated_From_Diocese',
    'Excardinated_To_Diocese',
    'Excardinated_To_Date',
    'Faculties',
    'Faculties_Granted_Date',
    'Faculties_Restricted_Date',
    'Faculties_Withdrawn_Date',
]

# These fields need to be KEPT but while building the SF upsert flow these are dropped temporarily until mapping logic is included.
# TODO

fields_not_yet_mapped = [
    'Common_Name',
    'Spouse',
    'Father_Full_Name',
    'Mother_Full_Maiden_Name',
    'Mailing_Address_Province',
    'Private_Address_Province',
    # 'Preferred_Address',
    # 'Private_Address__Street__s',
    # 'Private_Address_2',
    # 'Private_Address__City__s',
    # 'Private_Address__StateCode__s',
    # 'Private_Address__PostalCode__s',
    # 'Private_Address__CountryCode__s',
    'Preferred_Email',
    'Preferred_Phone',
    'Social_Security_Account_Number__c',  # The data is encrypted
    'Serving_Now',
    'Ordination_Diocese',
    'Registered_Parish'

]

In [ ]:
# UDF to combine multiple Mailing Street Address lines into one
def combine_addresses(row, *columns):
    address_parts = []
    for col in columns:
        value = row[col]
        if pd.notnull(value):  # Check for non-null values
            address_parts.append(str(value))  # Convert to string
    return '\n'.join(address_parts)  # '\n' for line break

In [ ]:
df_contact_staging = (df_contacts
                      .drop(columns='Salutation')
                      .rename(columns={
                          'Clergy_Status' : 'ADPDX_Clergy_Status__c',
                          'Religious_Status' : 'ADPDX_Religious_Status__c',
                          'Login_ID' : 'ADPDX_Login_ID__c',
                          'Access_Permission': 'ADPDX_Access_Permission__c',
                          'Title': 'Salutation',
                          'Christian_Name': 'FirstName',
                          'Middle_Name(s)': 'MiddleName',
                          'Surname': 'LastName',
                          'Suffix': 'Suffix',
                          'Preferred_Address': 'Preferred_Address__c',
                          'Mailing_Address_City': 'MailingCity',
                          'Mailing_Address_State': 'MailingState',
                          'Mailing_Address_Postal_Code': 'MailingPostalCode',
                          'Mailing_Address_Country': 'MailingCountry',
                          'Private_Address_City': 'OtherCity',
                          'Private_Address_State': 'OtherState',
                          'Private_Address_Postal_Code': 'OtherPostalCode',
                          'Private_Address_Country': 'OtherCountry',
                          'Work_Phone': 'npe01__WorkPhone__c',
                          'Home_Phone': 'HomePhone',
                          'Cell_Phone': 'MobilePhone',
                        #   'Preferred_Phone': 'npe01__PreferredPhone__c',
                          # IF Preferred phone contains, 'do not publish'
                          'Work_Email' : 'npe01__WorkEmail__c',
                          'Archdiocesan_Email': 'npe01__AlternateEmail__c',
                          'Home_Email': 'npe01__HomeEmail__c',
                        #   'Preferred_Email': 'npe01__Preferred_Email__c',
                          # IF Preferred email contains 'do not publish''
                          'Directory_Include': 'Directory_Include__c',
                          'Directory_Include_Middle_Name': 'Directory_Include_Middle_Name__c',
                          'Directory_Include_Suffix': 'Directory_Include_Suffix__c',
                          'Suppress_From_Reports': 'Suppress_From_Reports__c',
                          'Send_Group_Mail_and_Email': 'Send_Group_Mail_and_Email__c',
                          'Birth_Date': 'Birthdate',
                          'Place_of_Birth': 'mbfc__Place_of_Birth__c',
                          'Foreign_Born': 'Foreign_Born__c',
                          'Foreign_Citizenship': 'Foreign_Citizenship__c',
                          'Immigration_Status': 'Immigration_Status__c',
                          'Passport/Visa_Expiration_Date': 'Passport_Visa_Expiration_Date__c',
                          'Social_Security_Account_Number': 'Social_Security_Account_Number__c',
                          'Deceased_Date': 'mbfc__Date_of_Death__c',
                          'Out_of_Diocese_Date': 'mbfc__Date_Left_Diocese__c', 
                          'CARA_Ethnicity': 'adpdx_CARA_Ethnicity__c',
                          'Seminarian_Status': 'adpdx_Seminarian_Status__c',
                          'Other_Diaconal_Ministry': 'adpdx_Other_Diaconal_Ministry__c',
                          'Spiritual_Director_Authorized': 'adpdx_Spiritual_Director_Authorized__c',
                          'Place_of_Work': 'adpdx_Place_of_Work__c',
                          'Volunteer_Place': 'adpdx_Volunteer_Place__c',
                          'Type_of_Work': 'adpdx_Type_of_Work__c',
                          'Work_Load': 'adpdx_Work_Load__c',
                          'Work_Title': 'adpdx_Work_Title__c',
                          'Coverage_Availability': 'adpdx_Coverage_Availability__c', 
                          'Advanced_Directive_Date': 'adpdx_Advanced_Directive_Date__c',
                          'End_of_Life_Plan_Date': 'adpdx_End_of_Life_Plan_Date__c',
                          'Will_Date': 'adpdx_Will_Date__c',
                          'Will_Note': 'adpdx_Will_Note__c',
                          'CIC_489_File': 'adpdx_CIC_489_File__c',
                          'Senior_Status_Date': 'adpdx_Senior_Status_Date__c', 
                          'Laicized_Date': 'adpdx_Laicized_Date__c',
                          'Seminarian_Student_Debt': 'adpdx_Seminarian_Student_Debt__c',
                          'Seminarian_Medical_Benefits': 'adpdx_Seminarian_Medical_Benefits__c',
                          'Candidacy_Date': 'adpdx_Candidacy_Date__c',
                          'Accepted_to_Formation_Date': 'adpdx_Accepted_to_Formation_Date__c',
                          'Formation_Withdrawn_Date': 'adpdx_Formation_Withdrawn_Date__c',
                          'Formation_Deferred_Date': 'adpdx_Formation_Deferred_Date__c',
                          'Formation_Terminated_Date': 'adpdx_Formation_Terminated_Date__c',
                          'Terminate_or_Defer_Note': 'adpdx_Terminate_or_Defer_Note__c',
                          'CARA_Highest_Ed_Level': 'adpdx_CARA_Highest_Ed_Level__c',
                          'Letter_of_Good_Standing_Date': 'adpdx_Letter_of_Good_Standing__c',
                          'Religious_In_Archdiocese_Date': 'mbfc__Date_of_Arrival_in_Diocese__c',
                          'Last_Retreat_Date': 'adpdx_Last_Retreat_Date__c',
                          'Last_Educ_Requirement_Date': 'adpdx_Last_Educ_Requirement_Date__c',
                          'Policy_Manual_Acknowledgement_Date': 'adpdx_Policy_Manual_Acknowledgement_Date__c',
                          'Harassment_Prevention_Course_Date': 'adpdx_Harassment_Prevention_Course_Date__c',
                          'Standards_of_Conduct_Date': 'adpdx_Standards_of_Conduct_Date__c',
                          'Last_Background_Check_Date': 'adpdx_Last_Background_Check_Date__c',
                          'Last_Child_Protection_Training_Date': 'adpdx_Last_Child_Protection_Training__c',
                          'Languages': 'Languages__c',
                          'Nickname': 'adpdx_Preferred_Name__c'

                          })
                      .assign(Bi_Ritual__c=lambda x: x['Type(s)'].str.contains('Biritual'))
                      .assign(Non_Latin_Rite__c=lambda x: x['Type(s)'].str.contains('Non-Latin Rite'))
                      .assign(adpdx_Discerner_Aspirant_for_Diaconate__c=lambda x: x['Type(s)'].str.contains('Diaconate'))
                      .assign(adpdx_Is_Seminarian__c=lambda x: x['Type(s)'].str.contains('Seminar'))
                      
                      .assign(Archdpdx_Migration_Id__c=lambda x: x.index)
                      .assign(MailingStreet=lambda x: x.apply(lambda row: combine_addresses(row, 'Mailing_Address', 'Mailing_Address_2'), axis=1))
                      .drop(columns=['Mailing_Address', 'Mailing_Address_2'])  # Optional: Drop original columns if not needed
                      .assign(OtherStreet=lambda x: x.apply(lambda row: combine_addresses(row, 'Private_Address', 'Private_Address_2'), axis=1))
                      .drop(columns=['Private_Address', 'Private_Address_2'])  # Optional: Drop original columns if not needed
                      .drop(columns=misc_columns_to_drop)
                      .drop(columns=affiliation_columns)
                      .drop(columns=fields_not_yet_mapped)

        )


In [ ]:
df_contact_staging.columns

In [ ]:
df_contact_staging.MailingStreet.sample(10)

### Languages

In [ ]:
# # Define a function to clean the 'languages' column

# import re
# def clean_languages(text):
#     if pd.isna(text):
#         return text
#     # Remove text inside parentheses
#     text = re.sub(r'\(.*?\)', '', text)
#     # Replace ' & ' or ' and ' with ';'
#     text = re.sub(r' & | and ', ';', text)
#     # Replace commas with semicolons
#     text = text.replace(',', ';')
#     # Remove spaces before and after semicolons
#     text = re.sub(r'\s*;\s*', ';', text)
#     return text.strip(';')

# # Apply the cleaning function to the 'languages' column
# df_contact_staging['Languages__c'] = df_contact_staging['Languages__c'].apply(clean_languages)


### Private Address Handling


In [ ]:
# If 'OtherStreet' is not null, then set Secondary Address Type to 'Private'.  This is because the 'OtherAddress' fields all come from the 'Private' address fields in source system. 
df_contact_staging['npe01__Secondary_Address_Type__c'] = df_contact_staging['OtherStreet'].apply(lambda x: 'Private' if pd.notnull(x) else None)


### Handle Boolean Fields


In [ ]:
boolean_columns_to_convert = ['Foreign_Born__c', 'Directory_Include__c', 'Directory_Include_Middle_Name__c', 'Directory_Include_Suffix__c',
       'Suppress_From_Reports__c', 'Send_Group_Mail_and_Email__c', ]

df_contact_staging[boolean_columns_to_convert] = df_contact_staging[boolean_columns_to_convert].replace({'Yes': True, 'No': False})


In [ ]:
df_contact_staging[boolean_columns_to_convert] = df_contact_staging[boolean_columns_to_convert].fillna(False)

df_contact_staging[boolean_columns_to_convert].sample(5)

### Set Contact Record Type


In [ ]:
# Set Record Type

# Go down row by row and check the 'Type(s)' columns, check for certain words that are keys in a dictionary, and
# the that row's 'Type(s)' field contains a string that is in the a key in a dictionary the update another columns
# called 'ContactRecordType' with the paired value.

contact_type_map = {
    'Bishop': 'Priest',
    'Priest': 'Priest',
    'Transitional Deacon': 'Permanent_Deacon',
    'Permanent Deacon': 'Permanent_Deacon',
    'Seminarian': 'Lay_Person',
    'Diaconate Formation': 'Lay_Person',
    'Seminary Applicant': 'Lay_Person',
    'Diaconate Inquirer': 'Lay_Person',
    'Wife': 'Lay_Person',
    'Religious': 'Religious',
    'Staff': 'Lay_Person',
    'Seminary Applicant': 'Lay_Person',
    'Archive': 'Lay_Person'
}

def update_contact_record_type(row):
    for key, value in contact_type_map.items():
        if key in row['Type(s)']:
            return value
    return None

df_contact_staging['ContactRecordType'] = df_contact_staging.apply(update_contact_record_type, axis=1)

In [ ]:
# Map in the RecordTypeIDs
df_contact_staging['RecordTypeID'] = df_contact_staging['ContactRecordType'].map(record_types_mapping)

### Ecclesial Status & Ministerial Status


In [ ]:
df_contact_staging

In [ ]:
def determine_ecclesial_status(df):
    def ecclesial_status(row):
        if pd.notna(row['ADPDX_Clergy_Status__c']) and 'Laicized' in row['ADPDX_Clergy_Status__c']:
            return 'Laicized'
        # elif pd.notna(row['ADPDX_Clergy_Status__c']) and 'Faculties Withdrawn' in row['ADPDX_Clergy_Status__c']:
        #     return 'Faculties Withdrawn'
        elif pd.notna(row['Type(s)']) and 'Bishop' in row['Type(s)']:
            return 'Bishop/Archbishop'
        elif pd.notna(row['Type(s)']) and 'Priest,Religious' in row['Type(s)']:
            return 'Priest - Religious'
        elif pd.notna(row['Type(s)']) and 'Priest' in row['Type(s)'] and (not pd.isna(row['Foreign_Citizenship__c']) or row['Incardinated_Now'] != 'Archdiocese of Portland in Oregon'):
            return 'Priest - Temporary Sojourn (Foreign)'
        elif pd.notna(row['Type(s)']) and 'Priest' in row['Type(s)'] and (pd.isna(row['Foreign_Citizenship__c']) and row['Incardinated_Now'] == 'Archdiocese of Portland in Oregon'):
            return 'Priest - Diocesan'
        elif pd.notna(row['Type(s)']) and row['Type(s)'] == 'Permanent Deacon':
            return 'Permanent Deacon'
        else:
            return None

    df['mbfc__Ecclesial_Status__c'] = df.apply(ecclesial_status, axis=1)
    return df


df_contact_staging = determine_ecclesial_status(df_contact_staging)

In [ ]:
def determine_ministerial_status(df):
    def ministerial_status(row):
        if row['ADPDX_Clergy_Status__c'] == 'Deceased':
            return 'Deceased'
        elif row['ADPDX_Clergy_Status__c'] == 'Active':
            return 'Active in Ministry'
        elif row['ADPDX_Clergy_Status__c'] == 'Inactive':
            return 'Inactive'
        elif row['ADPDX_Clergy_Status__c'] == 'Senior Status':
            return 'Senior Status'
        elif row['ADPDX_Clergy_Status__c'] == 'Faculties Withdrawn':
            return 'Faculties Withdrawn'
        elif row['ADPDX_Clergy_Status__c'] == 'Transferred Out':
            return 'Left Diocese'
        elif row['ADPDX_Clergy_Status__c'] == 'Unassigned':
            return 'Unassigned'
        elif row['ADPDX_Clergy_Status__c'] == 'Laicized':
            return 'Laicized'
        else:
            return 'Unknown'
        
    df['mbfc__Ministerial_Status__c'] = df.apply(ministerial_status, axis=1)
    return df

df_contact_staging = determine_ministerial_status(df_contact_staging)

### Religious Congregation
In this section, for those Contacts who have a value in the `Link to Religious Community` source field we need to populate the `mbfc__Religious_Order__c` target field in Salesforce with the correct Religious Community's parent account - the Religious Congregation.

NOTE: In the source data, there is no differentiation between a child Religious Community and a parent Religious Order, there is only one record for the Religious Comnmunity.  In MF360 we represent these Accounts separately so we need to first (a) get the Religious Community record using the `Link to Religious Community` value but transforming it (adding 'RelCommunities_' in front of the value) so it matches the Archdpdx_Migration_Id__c in Salesforce.  

Once acquired, (b) we need to get the value of the `ParentID` field on the Religious Community which is the ID of the Religious Congregation record.  That ID is the value we then want to populate in the `mbfc__Religious_Order__c` field. 

In [ ]:
# get SF Account
get_all_accounts = 'Select Id, Name, RecordTypeId, Type, mbfc__Parish_Code__c, Job_Id__c, Archdpdx_Migration_Id__c, ParentID from Account WHERE Archdpdx_Migration_Id__c != null'

# get list of records, add to dataframe
sf_accounts = sf.query(get_all_accounts)
df_sf_accounts = pd.DataFrame(sf_accounts['records'])
df_sf_accounts = df_sf_accounts.drop(columns = 'attributes')

# create a dict in order to apply later
accounts_id_map = df_sf_accounts.set_index('Archdpdx_Migration_Id__c')['Id'].to_dict()

In [ ]:
df_sf_accounts[df_sf_accounts['Archdpdx_Migration_Id__c'].str.contains('RelCommunities', na=False)]

In [ ]:
# applies a lambda function to each element in the ‘Link_to_Religious_Community’ column, prefixing the value with 'RelCommunities_'
def transform_religious_community_link(df):
    df['Link_to_Religious_Community'] = df['Link_to_Religious_Community'].apply(
        lambda x: None if x == '0' else f'RelCommunities_{x}'
    )
    return df

# This function searches for a record in the sf_accounts DataFrame where the ‘Archdpdx_Migration_Id__c’ column matches the given archdpdx_migration_id
def get_parent_id_from_salesforce(sf_accounts, archdpdx_migration_id):
    print(f"Searching for: {archdpdx_migration_id}")  # Debug print
    matching_record = sf_accounts[sf_accounts['Archdpdx_Migration_Id__c'] == archdpdx_migration_id]
    if not matching_record.empty:
        print(f"Found: {matching_record['ParentId'].values[0]}")  # Debug print
        return matching_record['ParentId'].values[0]
    print("Not found")  # Debug print
    return None

# uses the get_parent_id_from_salesforce function to find the ‘ParentId’ from the sf_accounts DataFrame
def update_religious_order(df, sf_accounts):
    df['mbfc__Religious_Order__c'] = df.apply(
        lambda row: get_parent_id_from_salesforce(sf_accounts, row['Link_to_Religious_Community']) 
        if row['Link_to_Religious_Community'] is not None else None, axis=1
    )
    return df


# run the transform_religious_community_link and update_religious_order functions
df_contact_staging = transform_religious_community_link(df_contact_staging)

df_contact_staging = update_religious_order(df_contact_staging, df_sf_accounts)

In [ ]:
df_contact_staging[df_contact_staging.mbfc__Religious_Order__c.isna() == False]

### Registered Parish

In this section we populate the 'Home Parish' target field for Contacts who have a 'Registered Parish' in the source system. 

TODO: Check to see if the Registered Parish data is worth importing. Currently, 'Registered Parish' is only populated on 51 rows, and 32 of those rows in the 'Types' field are listed as 'Archive'. In other words, **only 19 of the 51 rows have a 'Registered Parish' value that might be meaningful.** 

### Diocese of Incardination

In [ ]:
df_contact_staging['Incardinated_Now'].sample(10)

In [ ]:
# Need to look for, then create a new Account that corresponds to a given 'Diocese of Incardination', then populate with record Id. 

# Assuming the function get_or_create_account is already defined as provided above

def update_incardinated_accounts(sf, df, column_name, record_type_dev_name, church_type, new_column_name):
    """
    Update the DataFrame by getting or creating Salesforce accounts for the values in the specified column.

    Parameters:
    sf (Salesforce): Salesforce connection object
    df (pd.DataFrame): The DataFrame to update
    column_name (str): The name of the column to search for account names
    record_type_dev_name (str): The developer name of the Record Type to use for creating the account
    church_type (str): The Church Type to set for the new account
    new_column_name (str): The name of the new column to store the Salesforce account IDs

    Returns:
    pd.DataFrame: The updated DataFrame with the new column containing Salesforce account IDs
    """
    df[new_column_name] = None

    for index, row in df.iterrows():
        account_name = row[column_name]
        if pd.notna(account_name):
            account_id = get_or_create_account(sf, account_name, record_type_dev_name, church_type)
            df.at[index, new_column_name] = account_id
    
    return df

# Example usage
# sf = Salesforce(username='your_username', password='your_password', security_token='your_security_token')
df_contact_staging = update_incardinated_accounts(sf, df_contact_staging, 'Incardinated_Now', 'Church', 'Diocese', 'mbfc__Diocese_of_Incardination__c')



In [ ]:
df_contact_staging[['mbfc__Diocese_of_Incardination__c', 'Incardinated_Now']].sample(20)

In [ ]:
# Drop the 'Incardinated Now' column 
del df_contact_staging['Incardinated_Now']


### Final Dataframe Cleanup


In [ ]:
# drop columns that are no longer needed
# del df_contact_staging['Type(s)']  # Commented this out as we want to KEEP the field and migrated to 'ADPDX Contact Type'
del df_contact_staging['ContactRecordType']
del df_contact_staging['Link_to_Religious_Community']

In [ ]:
df_contact_staging = df_contact_staging.rename(columns={'Type(s)': 'ADPDX_Contact_Type__c'})

In [ ]:
# convert '' to NaN
df_contact_staging.replace("", np.nan, inplace=True)

# convert NaN to None
df_contact_staging = df_contact_staging.where(df_contact_staging.notnull(), None)


In [ ]:
df_contact_staging['Languages__c'].sample(20)

In [ ]:
# df_contact_staging_2 = df_contact_staging.where(df_contact_staging.notnull(), None)

## Load


In [ ]:
df_contact_staging['Archdpdx_Job_Id__c'] = curr_job_id

In [ ]:
# generate CSV for manual loading
df_contact_staging.to_csv(f'/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/df_contacts_staging.csv', encoding='utf-8-sig')
df_contact_staging.to_csv('staging_files/contacts_staging.csv', encoding='utf-8-sig')


In [ ]:
# upsert Contact records into SF using Bulk api

from simple_salesforce.exceptions import SalesforceMalformedRequest

bulk_data = []
for row in df_contact_staging.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

try:
    # Attempt to upsert Contact records into SF using Bulk API
    contact_upsert = sf.bulk.Contact.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=500, use_serial=False)
    contact_upsert_results = pd.DataFrame(contact_upsert)
except SalesforceMalformedRequest as e:
    # If a SalesforceMalformedRequest error occurs, print the error message and response content
    print(f"SalesforceMalformedRequest error: {e}")
    print(f"Response content: {e.content}")



In [ ]:
# Print upsert results to local file

keys = contact_upsert[0].keys()
with open('results_files/contact_results', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, keys)
    writer.writeheader()
    writer.writerows(contact_upsert)


# CONTACT > SPOUSES

#TODO: Contact Spouses migration


# CONTACTS > PHOTOS

#TODO: Contact Photos


# CONTACT > REGISTER ENTRIES


In [ ]:
import pandas as pd

# Load CSV
df = (pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/People.csv')
               .rename(columns=lambda x: x.replace(' ', '_')) # Remove whitespace in column names
               .drop(index=0) # Drops the extra row that replicates the labels
)

df

In [ ]:
# Import all Contact fields that actually map to Register Entry records

import pandas as pd

# Define the structure of your column sets with correct attribute names
column_sets = [
    {'date': 'Baptism_Date', 'place': 'Place_of_Baptism', 'notation_type': 'Proof of Baptism'},
    {'date': 'Confirmation_Date', 'place': 'Place_of_Confirmation', 'notation_type': 'Notice of Confirmation'},
    {'date': 'Received_Date', 'place': 'Parish_of_Record', 'notation_type': 'Notice of Profession of Faith'},
    {'date': 'Marriage_Date', 'place': 'Place_of_Marriage', 'notation_type': 'Notice of Matrimony'},
    {'date': 'Diaconal_Ordination_Date', 'place': 'Diaconal_Ordination_Place', 'prelate': 'Diaconate_Ordination_Prelate', 'notation_type': 'Notice of Holy Orders', 'ordination_type': 'Diaconate'},
    {'date': 'Presbyteral_Ordination_Date', 'place': 'Presbyteral_Ordination_Place', 'prelate': 'Presbyteral_Ordination_Prelate', 'notation_type': 'Notice of Holy Orders', 'ordination_type': 'Presbyteral'},
    {'date': 'Episcopal_Ordination_Date', 'place': 'Episcopal_Ordination_Place', 'prelate': 'Episcopal_Ordination_Prelate', 'notation_type': 'Notice of Holy Orders', 'ordination_type': 'Episcopal'}
]

# New DataFrame for entries
register_entries = pd.DataFrame(columns=['RecordNumber', 'mbfc__Register_Entry_Type__c', 'mbfc__Type__c', 'mbfc__Notation_Type__c', 'mbfc__Ordination_Type__c', 'Date', 'Place', 'Prelate'])
new_entries = []  # List to store entries before final concatenation

# Processing rows
for row in df.itertuples():
    for column_set in column_sets:
        date_value = getattr(row, column_set['date'], None)
        if pd.notna(date_value):  # Check if date field is not NaN
            entry = {
                'RecordNumber': getattr(row, 'Record_Number', None),
                'Date': date_value,
                'Place': getattr(row, column_set['place'], None)
            }
            # Add Prelate if applicable
            if 'prelate' in column_set:
                entry['Prelate'] = getattr(row, column_set['prelate'], None)

            # Set 'mbfc__Register_Entry_Type__c', and conditionally add 'mbfc__Type__c' or 'mbfc__Notation_Type__c'
            if 'sacrament_type' in column_set:
                entry['mbfc__Type__c'] = column_set['sacrament_type']
                entry['mbfc__Register_Entry_Type__c'] = 'Sacrament'
            if 'notation_type' in column_set:
                entry['mbfc__Notation_Type__c'] = column_set['notation_type']
                entry['mbfc__Register_Entry_Type__c'] = 'Notation'

            # Handle ordination type specific updates
            if 'ordination_type' in column_set:
                entry['mbfc__Ordination_Type__c'] = column_set['ordination_type']

            new_entries.append(entry)
    
    # Add entries for 'Reader Date'
    reader_date = getattr(row, 'Reader_Date', None)
    if pd.notna(reader_date):
        entry = {
            'RecordNumber': getattr(row, 'Record_Number', None),
            'Date': reader_date,
            'mbfc__Notation_Type__c': 'Notice of Holy Orders',
            'mbfc__Ordination_Type__c': 'Minor Order: Reader',
            'mbfc__Register_Entry_Type__c': 'Notation'
        }
        new_entries.append(entry)
    
    # Add entries for 'Acolyte Date'
    acolyte_date = getattr(row, 'Acolyte_Date', None)
    if pd.notna(acolyte_date):
        entry = {
            'RecordNumber': getattr(row, 'Record_Number', None),
            'Date': acolyte_date,
            'mbfc__Notation_Type__c': 'Notice of Holy Orders',
            'mbfc__Ordination_Type__c': 'Minor Order: Acolyte',
            'mbfc__Register_Entry_Type__c': 'Notation'
        }
        new_entries.append(entry)

# Concatenate all new entries to the DataFrame at once
if new_entries:
    register_entries = pd.concat([register_entries, pd.DataFrame(new_entries)], ignore_index=True)

print(f"Total records added: {len(register_entries)}")

# Optionally, save the new DataFrame to a CSV
register_entries.to_csv('Register_Entries.csv', index=False)

# Display the DataFrame
register_entries.sample(10)


### Populate Lookup for Prelate 

In [ ]:
from nameparser import HumanName
from nameparser.config import CONSTANTS

# Add dataset-specific Titles and Suffix constants for parsing
CONSTANTS.titles.add('Very', 'Rev.', 'Very Rev.', 'Sr.', 'Most Rev.')
CONSTANTS.suffix_acronyms.add('FRS', 'J.C.L.', 'J.C.L., D.D.', 'D.D.', 'OMI', 'OSA', 'OCD', 'OP', 'OC', 'FSE', 'OMV', 'SDB', 'SM', 'SFX', 'SP', 'OP', 'O.S.M', 'SNJM', 'OSF', 'HMRF', 'DD', 'CSJP', 'SDD', 'BVM', 'BVM - President', 'SJ', 'SL', 'IX', 'SSJ', 'J.C.L.', 'J.C.L', 'OFM', 'MSpS', 'Fco.' )


def parse_name(name):
    if pd.isna(name):  # Checks if the name is NaN or None
        return {
            'Salutation': '',
            'FirstName': '',
            'MiddleName': '',
            'LastName': '',
            'Suffix': ''
        }
    else:
        name = HumanName(name)
        return {
            'Salutation': name.title,
            'FirstName': name.first,
            'MiddleName': name.middle,
            'LastName': name.last,
            'Suffix': name.suffix
        }

# Apply the parsing function only where 'Prelate' exists and is not NaN
for entry in new_entries:
    if 'Prelate' in entry and pd.notna(entry['Prelate']):
        parsed_name = parse_name(entry['Prelate'])
        entry.update(parsed_name)

# Ensure the DataFrame creation from new_entries includes checks for existence of keys:
register_entries = pd.DataFrame(new_entries)
if 'Prelate' in register_entries.columns:
    register_entries['Salutation'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['Salutation'] if pd.notna(x) else '')
    register_entries['FirstName'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['FirstName'] if pd.notna(x) else '')
    register_entries['MiddleName'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['MiddleName'] if pd.notna(x) else '')
    register_entries['LastName'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['LastName'] if pd.notna(x) else '')
    register_entries['Suffix'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['Suffix'] if pd.notna(x) else '')


# Display the DataFrame
print(f"Total records added: {len(register_entries)}")
register_entries.sample(10)



In [ ]:
# Query Salesforce for existing contacts and create a dictionary for mapping

from simple_salesforce import Salesforce

query = """
SELECT Id, Archdpdx_Migration_Id__c
FROM Contact
"""
result = sf.query_all(query)
contact_map = {rec['Archdpdx_Migration_Id__c']: rec['Id'] for rec in result['records']}


In [ ]:
# Get RecordTypeId for Contact.Priest

priest_contact_recordtype_id = df_sf_recordTypes.loc[
    (df_sf_recordTypes['DeveloperName'] == 'Priest') & (df_sf_recordTypes['SobjectType'] == 'Contact'),
    'Id'
    ].iloc[0]  # Use .iloc[0] to get the first item if you're expecting exactly one match


In [ ]:
# Get RecordID for Prelates by querying for Contacts by FirstName and LastName and, if not found, Create New Contacts

from simple_salesforce import SFType, SalesforceResourceNotFound

contact = SFType('Contact', sf.session_id, sf.sf_instance)
for index, row in register_entries.iterrows():
    first_name, last_name = row.get('FirstName'), row.get('LastName')

    if pd.isna(first_name) or pd.isna(last_name) or first_name.strip() == '' or last_name.strip() == '':
        # If either first name or last name is missing or empty, skip this row or handle as needed
        print(f"Skipping row {index} due to missing name information.")
        continue

    try:
        # Search for contact by First and Last Name
        query = f"SELECT Id FROM Contact WHERE FirstName = '{first_name}' AND LastName = '{last_name}'"
        result = sf.query(query)
        if result['totalSize'] > 0:
            contact_id = result['records'][0]['Id']
        else:
            # Create a new contact if no match found
            new_contact = {
                'FirstName': first_name,
                'LastName': last_name,
                'Archdpdx_Job_Id__c': curr_job_id,
                'RecordTypeId': priest_contact_recordtype_id
            }
            create_result = contact.create(new_contact)
            contact_id = create_result['id']

        # Update DataFrame with the Salesforce Contact ID
        register_entries.at[index, 'mbfc__Celebrant__c'] = contact_id

    except SalesforceException as e:
        print(f"Error processing row {index}: {e}")



### Prepare to Upsert   

In [ ]:
# Map Contact IDs to Register Entries

register_entries_2 = register_entries

register_entries_2['mbfc__Contact__c'] = register_entries['RecordNumber'].map(contact_map)


In [ ]:
# Append Job_Id__c
register_entries_2['Archdpdx_Job_Id__c'] = curr_job_id

In [ ]:
# Generate an External ID
def create_external_id(row):
    record_number = str(row['RecordNumber']).replace(' ', '').replace('-', '')
    entry_type = str(row['mbfc__Register_Entry_Type__c']).replace(' ', '').replace('-', '')

    # Check whether to use Type or Notation Type based on what's available
    if 'mbfc__Type__c' in row and not pd.isna(row['mbfc__Type__c']):
        type_field = str(row['mbfc__Type__c']).replace(' ', '').replace('-', '')
    elif 'mbfc__Notation_Type__c' in row and not pd.isna(row['mbfc__Notation_Type__c']):
        type_field = str(row['mbfc__Notation_Type__c']).replace(' ', '').replace('-', '') + str(row['mbfc__Ordination_Type__c']).replace(' ', '').replace('-', '')
    else:
        type_field = 'Unknown'

    return f"{record_number}_{entry_type}_{type_field}"

In [ ]:
# Assuming your DataFrame is named `register_entries`
register_entries_2['Archdpdx_Migration_Id__c'] = register_entries.apply(create_external_id, axis=1)

if register_entries['Archdpdx_Migration_Id__c'].duplicated().any():
    print("Warning: There are duplicate external IDs.")
    # Optionally, show the duplicates
    duplicates = register_entries[register_entries['external_id'].duplicated(keep=False)]
    print(duplicates)
else:
    print("All external IDs are unique.")


In [ ]:
# Drop unnecessary columns:
register_entries_2.drop(['RecordNumber', 'Prelate', 'Salutation', 'FirstName', 'MiddleName', 'LastName', 'Suffix'], axis=1, inplace=True)

In [ ]:
register_entries_staging = register_entries_2

In [ ]:
# Remove all NaN values:
register_entries_staging.fillna('', inplace=True)

# Rename columns
register_entries_staging = register_entries_staging.rename(columns={
    'Place': 'mbfc__Location_text__c',
    'Date': 'mbfc__Event_Date__c'
})


In [ ]:
# What is this checking for?... Why did I include this?
register_entries_staging[register_entries_staging.mbfc__Contact__c == '003Dx00000m0OtXIAU']


In [ ]:
# generate CSV for manual loading
register_entries_staging.to_csv('staging_files/reg_entry_staging.csv', encoding='utf-8-sig')


In [ ]:
# Upsert Register Entry Records

bulk_data = []
for row in register_entries_staging.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

# Keep the batch <100 as I've been getting an exceptionCode: 'InvalidBatch', 'exceptionMessage': 'Records not processed'
reg_entry_upsert = sf.bulk.mbfc__Register_Entry__c.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
reg_entry_upsert_results = pd.DataFrame(reg_entry_upsert)

In [ ]:
# Print upsert results to local file

keys = reg_entry_upsert[0].keys()

with open('results_files/register_entry_results', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, keys)
    writer.writeheader()
    writer.writerows(reg_entry_upsert)

# CONTACT > AFFILIATIONS


In [ ]:
# Function to create a unique ID based on Person's Name + completion date + affiliation type
def create_unique_id(row):
    # Concatenate the three fields with mbfc__Person__c at the front
    combined = f"{row['mbfc__Person__c']}{row['mbfc__Completion_Date__c']}{row['mbfc__Affiliation__c']}"
    # Remove unwanted characters and convert to lowercase
    clean_id = ''.join(combined.split()).replace('-', '').replace('.', '').lower()
    # Limit the string to 50 characters
    return clean_id[:50]

## Education Affiliations

This section takes multiple sets of columns (all related to a person's education) from the Contacts table, and combines them into a single set of columns in a new dataframe for insertion into Salesforce as Affiliation records.


In [ ]:
# Parse and stage Education Affiliation records
import pandas as pd
from functools import lru_cache

# Load CSV
df = (pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/People.csv')
               .rename(columns=lambda x: x.replace(' ', '_')) # Remove whitespace in column names
               .drop(index=0) # Drops the extra row that replicates the labels
)


# Define the structure of your column sets with correct attribute names
degree_sets = [
    {'year': 'Bachelor_Degree_Year', 'type': 'Bachelor_Degree_Type', 'institution': 'Bachelor_Degree_Institution'},
    {'year': 'Graduate_1_Degree_Year', 'type': 'Graduate_1_Degree_Type', 'institution': 'Graduate_1_Degree_Institution'},
    {'year': 'Graduate_2_Degree_Year', 'type': 'Graduate_2_Degree_Type', 'institution': 'Graduate_2_Degree_Institution'},
    {'year': 'Graduate_3_Degree_Year', 'type': 'Graduate_3_Degree_Type', 'institution': 'Graduate_3_Degree_Institution'},
    {'year': 'Graduate_4_Degree_Year', 'type': 'Graduate_4_Degree_Type', 'institution': 'Graduate_4_Degree_Institution'}
]

# Query for the Record Type ID for 'Organization'
record_type_result = sf.query("SELECT Id FROM RecordType WHERE SobjectType = 'Account' AND DeveloperName = 'Organization'")
organization_record_type_id = record_type_result['records'][0]['Id'] if record_type_result['records'] else None

# Initialize the DataFrame for the staging table
education_staging = pd.DataFrame()

# Function to check and create institution account
@lru_cache(maxsize=None)
def get_or_create_institution_account(institution_name):
    if pd.isna(institution_name):
        return None  # Return None or handle as appropriate if institution name is NaN

    # Query Salesforce to find the institution
    query = f"SELECT Id, Name FROM Account WHERE Name = '{institution_name}' LIMIT 1"
    results = sf.query(query)
    
    # If exists, return the ID
    if results['records']:
        return results['records'][0]['Id']
    else:
        # Ensure no NaN values are sent to Salesforce
        account_data = {
            'Name': institution_name if pd.notna(institution_name) else "Default Name",  # Provide a default if NaN
            'RecordTypeId': organization_record_type_id,
            'mbfc__Organization_Type__c': 'School'
        }
        # Remove keys with None values to avoid JSON serialization issues
        account_data = {k: v for k, v in account_data.items() if v is not None}
        
        new_account = sf.Account.create(account_data)
        return new_account['id']

# Get Contact record ID from Salesforce
@lru_cache(maxsize=None)
def get_contact_id_by_record_number(record_number):
    if pd.isna(record_number):
        return None
    query = f"SELECT Id FROM Contact WHERE Archdpdx_Migration_Id__c = '{record_number}'"
    results = sf.query(query)
    if results['records']:
        return results['records'][0]['Id']
    return None


# Initialize an empty list to collect DataFrames or dictionaries
new_entries = []

# Process each row and each degree set
for index, row in df.iterrows():
    for degree_set in degree_sets:
        year = row[degree_set['year']]
        if pd.notna(year):  # Only proceed if the year column is not NaN
            formatted_year = f"{int(year)}-01-01"  # Convert year to YYYY-MM-DD format
            institution_name = row[degree_set['institution']]
            account_id = get_or_create_institution_account(institution_name)
            contact_id = get_contact_id_by_record_number(row['Record_Number'])
            
            # Create a record for the staging table
            affiliation_record = {
                'mbfc__Person__c': contact_id,
                'mbfc__Completion_Date__c': formatted_year,
                'mbfc__Context__c': account_id,
                'mbfc__Category__c': 'Education/Studies',
                'mbfc__Affiliation__c': row[degree_set['type']]
            }
            new_entries.append(affiliation_record)

# Convert all collected records to a DataFrame in one go
education_staging = pd.DataFrame(new_entries)


#FIXME: There are 4 rows where no INSTITUTION is listed. This makes it impossible to import an Affiliation record. Need to figure out how to handle this with Client. 
#FIXME: There are about 15 rows where no DEGREE is listed. This makes it impossible to import an Affiliation record. Need to figure out how to handle this with Client. 

In [ ]:
# Apply the function to each row and create a new column with the unique ID
education_staging['Archdpdx_Migration_Id__c'] = education_staging.apply(create_unique_id, axis=1)

# Check the first few rows to verify the new column
education_staging.head()

In [ ]:
# Fill any NaN values
education_staging = education_staging.fillna('')

In [ ]:
# Save the staging table to CSV
education_staging.to_csv('staging_files/education_staging.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
from simple_salesforce import Salesforce, SalesforceMalformedRequest, SalesforceError
from datetime import datetime, date

def convert_non_serializables(data):
    """Convert non-serializable objects to serializable formats."""
    for key, value in data.items():
        try:
            if isinstance(value, (datetime, date)):
                data[key] = value.isoformat()
            elif isinstance(value, float) and np.isnan(value):
                data[key] = None
            elif pd.isna(value):
                data[key] = None
            elif isinstance(value, (int, bool, str)):
                data[key] = value
            else:
                data[key] = str(value)  # Convert other types to string
        except Exception as e:
            print(f"Error processing key: {key}, value: {value}, error: {e}")
    return data


def upsert_to_salesforce(sf, dataframe, object_name, external_id_field):
    """
    Upsert records to Salesforce from a pandas DataFrame.

    Parameters:
    sf (Salesforce): The Salesforce connection instance.
    dataframe (pd.DataFrame): The pandas DataFrame containing data to upsert.
    object_name (str): The Salesforce object name (e.g., 'Contact').
    external_id_field (str): The external ID field used for upserts.
    """
    successful_upserts = 0
    failed_upserts = 0

    # Replace placeholder values with None in the DataFrame
    dataframe.replace({None: pd.NA, ' ': None, '': None}, inplace=True)

    # Convert DataFrame to a list of dictionaries
    data_to_upsert = dataframe.to_dict(orient='records')

    for data in data_to_upsert:
        try:
            data = convert_non_serializables(data)
            external_id = data.pop(external_id_field)

            # Perform upsert using only the External ID
            response = getattr(sf, object_name).upsert(f'{external_id_field}/{external_id}', data)
            successful_upserts += 1
            print(f"Successfully upserted {object_name} with External ID: {external_id}")
        except SalesforceMalformedRequest as e:
            failed_upserts += 1
            print(f"Malformed request error when upserting {object_name} with External ID: {external_id}. Error: {e.content}")
        except SalesforceError as e:
            failed_upserts += 1
            print(f"Salesforce error when upserting {object_name} with External ID: {external_id}. Error: {e.content}")
        except Exception as e:
            failed_upserts += 1
            print(f"Failed to upsert {object_name} with External ID: {external_id}. Error: {e}")

    print(f"Upsert completed. Successful upserts: {successful_upserts}, Failed upserts: {failed_upserts}")

def upsert_to_salesforce_bulk(sf, dataframe, object_name, external_id_field, failed_log_file, batch_size=10000):
    """
    Upsert records to Salesforce from a pandas DataFrame using the Bulk API.

    Parameters:
    sf (Salesforce): The Salesforce connection instance.
    dataframe (pd.DataFrame): The pandas DataFrame containing data to upsert.
    object_name (str): The Salesforce object name (e.g., 'Contact').
    external_id_field (str): The external ID field used for upserts.
    failed_log_file (str): The file name where failed upsert records will be logged.
    batch_size (int): The number of records to include in each batch.
    """
    successful_upserts = 0
    failed_upserts = 0

    # Replace placeholder values with None in the DataFrame
    dataframe.replace({None: pd.NA, ' ': None, '': None}, inplace=True)

    # Convert DataFrame to a list of dictionaries
    data_to_upsert = dataframe.to_dict(orient='records')

    with open(failed_log_file, 'a') as log_file:
        # Process data in batches
        for i in range(0, len(data_to_upsert), batch_size):
            batch_data = data_to_upsert[i:i + batch_size]
            batch_data = [convert_non_serializables(record) for record in batch_data]

            try:
                # Perform bulk upsert
                response = sf.bulk.__getattr__(object_name).upsert(batch_data, external_id_field=external_id_field)

                for res in response:
                    if res['success']:
                        successful_upserts += 1
                    else:
                        failed_upserts += 1
                        log_file.write(f"Failed to upsert record: {res}\n")

            except SalesforceMalformedRequest as e:
                failed_upserts += len(batch_data)
                log_file.write(f"Malformed request error when upserting batch. Error: {e.content}\n")
            except SalesforceError as e:
                failed_upserts += len(batch_data)
                log_file.write(f"Salesforce error when upserting batch. Error: {e.content}\n")
            except Exception as e:
                failed_upserts += len(batch_data)
                log_file.write(f"Failed to upsert batch. Error: {e}\n")

    print(f"Upsert completed. Successful upserts: {successful_upserts}, Failed upserts: {failed_upserts}")


In [ ]:
# Upsert Education Affiliation records

# upsert_to_salesforce(sf, education_staging, 'mbfc__Affiliation__c', 'Archdpdx_Migration_Id__c')
upsert_to_salesforce_bulk(sf, education_staging, 'mbfc__Affiliation__c', 'Archdpdx_Migration_Id__c', 'results_files/education_affil', batch_size=1000)


In [ ]:

#FIXME: A number of Education Affiliation records are missing either an Affiliation title or a Context

In [ ]:
# Upsert Education Affiliation records [DEP]

# bulk_data = []
# for row in education_staging.itertuples(index=False):
#     d = row._asdict()
#     # del d['Index']
#     bulk_data.append(d)

# try:
#     # Attempt to upsert Education Affiliation records into SF using Bulk API
    # education_affil_upsert = sf.bulk.mbfc__Affiliation__c.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=500, use_serial=False)
    

# except SalesforceMalformedRequest as e:
#     # If a SalesforceMalformedRequest error occurs, print the error message and response content
#     print(f"SalesforceMalformedRequest error: {e}")
#     print(f"Response content: {e.content}")

# Send results to CSV
# education_affil_upsert_results = pd.DataFrame(education_affil_upsert)
# education_affil_upsert_results.to_csv('results_files/education_affil_upsert_results')

## Ecclesial Affiliations

This section handles Contact table fields that map to Affiliation records with record type 'Ecclesial Affiliation'.

These Ecclesial Affiliations can be subcategorized by the 'contexts' to which the Affiliation records is related:

| Affiliation            | Context                   | Completion Date           |
| ---------------------- | ------------------------- | ------------------------- |
| First Vows             | Religious Order           | Date of First Vows        |
| Final Vows             | Religious Order           | Date of Final Vows        |
| Incardination          | Incardinated from Diocese | Incardinated From Date    |
| Faculties (Type)       | Local Diocese             | Faculties Granted Date    |
| Faculties (Restricted) | Local Diocese             | Faculties Restricted Date |
| Faculties (Withdrawn)  | Local Diocese             | Faculties Withdrawn Date  |
| Excardinated           | Excardinated To Diocese   | Excardinated To Date      |

FIXME: There are a number of rows where a Faculties Granted is missing a date, and conversely, where there is a Faculties Granted Date but no description of the Faculties granted. This is a problem, because the application requires a date for when Faculties were granted.


In [ ]:
# Generate a staging DF of Ecclesial Affiliations out of a handful of fields in the source data, each of which is to be converted into a new row in the staging DF.

import pandas as pd
from functools import lru_cache
from simple_salesforce import Salesforce

# Load CSV
df = (pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/People.csv')
               .rename(columns=lambda x: x.replace(' ', '_')) # Remove whitespace in column names
               .drop(index=0) # Drops the extra row that replicates the labels
)

# Define the structure of your column sets with correct attribute names
column_sets = [
    {'year': 'Incardinated_From_Date', 'context': 'Incardinated_From_Diocese'},
    {'year': 'Excardinated_To_Date', 'context': 'Excardinated_To_Diocese'},
    {'year': 'Faculties_Granted_Date', 'affiliation': 'Faculties'},
    {'year': 'Faculties_Restricted_Date'},
    {'year': 'Faculties_Withdrawn_Date'},
]

# Query for the Record Type IDs
record_type_query = "SELECT Id, DeveloperName FROM RecordType WHERE SobjectType = 'Account' AND DeveloperName IN ('Church', 'Religious')"
record_type_result = sf.query(record_type_query)
record_type_ids = {record['DeveloperName']: record['Id'] for record in record_type_result['records']}

church_record_type_id = record_type_ids.get('Church')
religious_record_type_id = record_type_ids.get('Religious')

# Query for the Record Type ID for 'Ecclesial Affiliation' for mbfc__Affiliation__c object
record_type_query = "SELECT Id FROM RecordType WHERE SobjectType = 'mbfc__Affiliation__c' AND DeveloperName = 'Ecclesial_Affiliation' LIMIT 1"
record_type_result = sf.query(record_type_query)

if not record_type_result['records']:
    raise ValueError("No RecordType found for Ecclesial Affiliation on mbfc__Affiliation__c object.")

ecclesial_affiliation_record_type_id = record_type_result['records'][0]['Id']

In [ ]:


# Initialize the DataFrame for the staging table
ecclesial_affiliation_staging = pd.DataFrame()

# Function to check and create institution account
@lru_cache(maxsize=None)
def get_or_create_church_account(context):
    if pd.isna(context):
        return None  # Return None or handle as appropriate if institution name is NaN

    # Query Salesforce to find the institution
    query = f"SELECT Id, Name FROM Account WHERE Name = '{context}' LIMIT 1"
    results = sf.query(query)
    
    # If exists, return the ID
    if results['records']:
        return results['records'][0]['Id']
    else:
        # Ensure no NaN values are sent to Salesforce
        if 'Diocese' in context or 'Archdiocese' in context:
            account_data = {
                'Name': context if pd.notna(context) else "Church Name Missing",  # Provide a default if NaN
                'RecordTypeId': church_record_type_id,
                'mbfc__Church_Type__c': 'Diocese'
            }
        else:
            account_data = {
                'Name': context if pd.notna(context) else "Religious Name Missing",  # Provide a default if NaN
                'RecordTypeId': religious_record_type_id
            }

        # Remove keys with None values to avoid JSON serialization issues
        account_data = {k: v for k, v in account_data.items() if v is not None}
        
        new_account = sf.Account.create(account_data)
        return new_account['id']

# Get Contact record ID from Salesforce
@lru_cache(maxsize=None)
def get_contact_id_by_record_number(record_number):
    if pd.isna(record_number):
        return None
    query = f"SELECT Id FROM Contact WHERE Archdpdx_Migration_Id__c = '{record_number}'"
    results = sf.query(query)
    if results['records']:
        return results['records'][0]['Id']
    return None

# Initialize an empty list to collect DataFrames or dictionaries
new_entries = []

# Process each row and each degree set
for index, row in df.iterrows():
    for col_set in column_sets:
        date = row[col_set['year']]
        if pd.notna(date):  # Only proceed if the year column is not NaN
            context = row.get(col_set.get('context'), None)
            account_id = get_or_create_church_account(context)
            contact_id = get_contact_id_by_record_number(row['Record_Number'])
            
            # Determine the mbfc__Affiliation__c value
            if 'Incardinated_From_Date' in col_set['year']:
                affiliation = 'Incardinated'
            elif 'Excardinated_To_Date' in col_set['year']:
                affiliation = 'Excardinated'
            elif 'Faculties_Granted_Date' in col_set['year']:
                faculties_value = row.get(col_set.get('affiliation', ''))
                if pd.isna(faculties_value):
                    affiliation = 'Faculties'
                else:
                    affiliation = f"Faculties ({faculties_value})"
                account_id = diocesan_account_id  # Override account ID for faculties
            elif 'Faculties_Restricted_Date' in col_set['year']:
                affiliation = 'Faculties (Restricted)'
                account_id = diocesan_account_id  # Override account ID for faculties
            elif 'Faculties_Withdrawn_Date' in col_set['year']:
                affiliation = 'Faculties (Withdrawn)'
                account_id = diocesan_account_id  # Override account ID for faculties
            elif 'Date_of_First_Vows' in col_set['year']:
                affiliation = 'First Vows'
            elif 'Date_of_Final_Vows' in col_set['year']:
                affiliation = 'Final Vows'
            else:
                affiliation = row.get(col_set.get('affiliation', ''), None)
            
            # Create a record for the staging table
            affiliation_record = {
                'RecordTypeId': ecclesial_affiliation_record_type_id,
                'mbfc__Person__c': contact_id,
                'mbfc__Completion_Date__c': date,
                'mbfc__Context__c': account_id,
                'mbfc__Category__c': 'Ecclesial Affiliations',
                'mbfc__Affiliation__c': affiliation
            }
            new_entries.append(affiliation_record)

# Convert all collected records to a DataFrame in one go
ecclesial_affiliations_staging = pd.DataFrame(new_entries)



In [ ]:
# Apply the function to each row and create a new column with the unique ID
ecclesial_affiliations_staging['Archdpdx_Migration_Id__c'] = ecclesial_affiliations_staging.apply(create_unique_id, axis=1)

# Check for duplicates
ecclesial_affiliations_staging['Archdpdx_Migration_Id__c'].duplicated().value_counts()

In [ ]:
# Send the new DataFrame to a CSV
ecclesial_affiliations_staging.to_csv('staging_files/Ecclesial_Affiliations_Staging.csv', index=False, encoding='utf-8-sig')

In [ ]:

# Display the DataFrame
ecclesial_affiliations_staging.sample(10)

In [ ]:
# NEW Upsert function to upsert Ecclesial Affiliation records
upsert_to_salesforce_bulk(sf, ecclesial_affiliations_staging, 'mbfc__Affiliation__c', 'Archdpdx_Migration_Id__c', 'results_files/ecclesial_affil_upsert_results')

In [ ]:
#FIXME: Handful of Ecclesial Affil records with error: [{'statusCode': 'FIELD_CUSTOM_VALIDATION_EXCEPTION', 'message': 'Context is required', 'fields': []}]"


# AFFILIATIONS


In [ ]:
# Import Assignments.csv

import pandas as pd


df_affiliations = (
    pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Assignments (1).csv')
    .set_index('Record Number', verify_integrity=True)
    .drop(index='recNum', errors='ignore')  # Added errors='ignore' to prevent errors if 'recNum' does not exist
    .drop(columns=['Historic Name'], errors='ignore')  # Added errors='ignore' for the same reason
    .rename(columns=lambda x: x.replace(' ', '_'))  # Remove whitespace in column names
    .assign(Account_Ext_Id=lambda df: df['Organization_Table_Name'] + '_' + df['Organization_Table_Link'])
    # .assign(mbfc__Person__r=lambda df: df['Assigned_Person'].apply(lambda x: {'Archdpdx_Migration_Id__c': x}))
    # .assign(mbfc__Context__r=lambda df: df['Account_Ext_Id'].apply(lambda x: {'Archdpdx_Migration_Id__c': x}))
    # .assign(mbfc__Use_Custom_Title__c= True)
    .assign(mbfc__Category__c= 'Any All')
    # .assign(Archdpdx_Migration_Id__c= df_affiliations.index)
    .drop(columns=[
        # 'Assigned_Person'
        'Organization_Table_Name'
        ,'Organization_Table_Link'
        ,'Projected_Term_End_Date'
        ,'Term_Number'
        ,'Leave_Type' # Leave out 'Leave_Type' until mapped properly
        ])
    .rename(columns={
        'Duty_Load': 'mbfc__Duty_Load__c',
        'Start_Date': 'mbfc__Start_Date__c',
        'End_Date': 'mbfc__Completion_Date__c',
        'Assignment_Title': 'mbfc__Affiliation__c',
        'Archdiocesan_Assignment': 'adpdx_Archdiocesan_Assignment__c',
    })
    .replace({'ADPDX_Archdiocesan_Assignment__c': {'Yes': True, 'No': False, None: False}})
    .fillna('')
)

# Display a sample of the DataFrame to check the new structure
df_affiliations.sample(10)



In [ ]:
# Get SF Record Ids from External Ids

# Get Context Account Ids
add_salesforce_record_ids(sf, df_affiliations, 'Account_Ext_Id', 'Account', 'Archdpdx_Migration_Id__c', 'mbfc__Context__c')

In [ ]:
# Get Person Contact Ids
add_salesforce_record_ids(sf, df_affiliations, 'Assigned_Person', 'Contact', 'Archdpdx_Migration_Id__c', 'mbfc__Person__c')

In [ ]:
# Set Archdpdx_Migration_Id__c External ID
df_affiliations['Archdpdx_Migration_Id__c'] = df_affiliations.index

# Create Job ID
df_affiliations['Archdpdx_Job_Id__c'] = curr_job_id

df_affiliations


In [ ]:
# Final cleanup
df_affiliations.drop(columns=[
    'Account_Ext_Id',
    'Assigned_Person', 
    ], 
    inplace=True)

df_affiliations

#FIXME: INVALID_FIELD: Foreign key external ID: relcommunities_23 not found for field Archdpdx_Migration_Id__c
#FIXME: INVALID_FIELD: Foreign key external ID: offices_0 not found for field Archdpdx_Migration_Id__c
#FIXME: Record #115 > FIELD_INTEGRITY_EXCEPTION: Start Date: invalid date: Tue Aug 01 00:00:00 GMT 1021 [mbfc__Start_Date__c

In [ ]:
df_affiliations.to_csv('staging_files/affiliations_staging.csv', encoding='utf-8', index=False)

In [ ]:
import pandas as pd
from simple_salesforce import Salesforce
from simple_salesforce.exceptions import SalesforceMalformedRequest, SalesforceError

def convert_non_serializables(record):
    """Convert non-serializable values to strings or handle them appropriately."""
    for key, value in record.items():
        if pd.isna(value):
            record[key] = None
        elif isinstance(value, pd.Timestamp):
            record[key] = value.isoformat()
        elif isinstance(value, (pd.Timedelta, pd.Period)):
            record[key] = str(value)
    return record

def upsert_to_salesforce_bulk(sf, dataframe, object_name, external_id_field, failed_log_file, batch_size=10000):
    """
    Upsert records to Salesforce from a pandas DataFrame using the Bulk API.

    Parameters:
    sf (Salesforce): The Salesforce connection instance.
    dataframe (pd.DataFrame): The pandas DataFrame containing data to upsert.
    object_name (str): The Salesforce object name (e.g., 'Contact').
    external_id_field (str): The external ID field used for upserts.
    failed_log_file (str): The file name where failed upsert records will be logged.
    batch_size (int): The number of records to include in each batch.
    """
    successful_upserts = 0
    failed_upserts = 0

    # Replace placeholder values with None in the DataFrame
    dataframe.replace({pd.NA: None, ' ': None, '': None}, inplace=True)

    # Convert DataFrame to a list of dictionaries
    data_to_upsert = dataframe.to_dict(orient='records')

    with open(failed_log_file, 'a') as log_file:
        # Process data in batches
        for i in range(0, len(data_to_upsert), batch_size):
            batch_data = data_to_upsert[i:i + batch_size]
            batch_data = [convert_non_serializables(record) for record in batch_data]

            try:
                # Perform bulk upsert
                response = sf.bulk.__getattr__(object_name).upsert(batch_data, external_id_field=external_id_field)

                for res in response:
                    if res['success']:
                        successful_upserts += 1
                    else:
                        failed_upserts += 1
                        log_file.write(f"Failed to upsert record: {res}\n")

            except SalesforceMalformedRequest as e:
                failed_upserts += len(batch_data)
                log_file.write(f"Malformed request error when upserting batch. Error: {e.content}\n")
                for record in batch_data:
                    log_file.write(f"Failed record: {record}\n")
            except SalesforceError as e:
                failed_upserts += len(batch_data)
                log_file.write(f"Salesforce error when upserting batch. Error: {e.content}\n")
                for record in batch_data:
                    log_file.write(f"Failed record: {record}\n")
            except Exception as e:
                failed_upserts += len(batch_data)
                log_file.write(f"Failed to upsert batch. Error: {e}\n")
                for record in batch_data:
                    log_file.write(f"Failed record: {record}\n")

    print(f"Upsert completed. Successful upserts: {successful_upserts}, Failed upserts: {failed_upserts}")

In [ ]:
upsert_to_salesforce_bulk(sf, 'mbfc__Affiliation__c', df_affiliations, 'Archdpdx_Migration_Id__c', 'results_files/affiliation_upsert_results')

In [ ]:
# @ Upsert Register Entry Records

bulk_data = []
for row in df_affiliations.itertuples(index=False):
    d = row._asdict()
    bulk_data.append(d)

In [ ]:
# Upsert Salesforce records
# FIXME: Encoding is getting messed up and I'm unsure how to pass in a parameter that will fix this. 

try:
    # Attempt to upsert Affiliation records into SF using Bulk API
    affiliation_upsert = sf.bulk.mbfc__Affiliation__c.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=1000, use_serial=False)
    affiliation_upsert_results = pd.DataFrame(affiliation_upsert)
    affiliation_upsert_results.to_csv('results_files/affiliation_upsert_results')

except SalesforceMalformedRequest as e:
    # If a SalesforceMalformedRequest error occurs, print the error message and response content
    print(f"SalesforceMalformedRequest error: {e}")
    print(f"Response content: {e.content}")


# Post-Migration Manual Updates

1. Convert 'Offices' that are ADPDX Pastoral Centre offices into record type: 'Groups', and set their parentID to the Diocese (there are just 6 of these accounts).
1. Update the Religous Order records 'Religious Superior' lookup.
1. Set 'organization type' field value for each account in the 'organization' load: Offices, Newman Centres, Schools, Organizations
1. Consolidate education degree titles in 'Affiliation.Affiliation' picklist into the standard value
